# Подбор моделей для этапа ранжирования кандидатов

### Описание
Для пакетной рекомендательной системы характерны 3 этапа работы:
1. Генерация кандидатов. На этом этапе происходит отбор сотен или тысяч объектов-кандидатов из множества объектов, которых могут быть миллиарды. Цель заключается в том, чтобы включить в этот набор макси-мально релевантные пользователю объекты и исключить нерелевантные.
2. Скоринг и ранжирование. Здесь система оценивает кандидатов, полученных на предыдущем этапе, и присваивает каждому из них баллы, основанные на их релевантности для пользователя. Т.к. количество объек-тов на этом этапе ограничено, то можно использовать больший набор при-знаков объектов, контекстные факторы, например время и геопозицию.
3. Повторное ранжирование. В рамках данного этапа проводится корректировка списка кандидатов, применяются бизнес-правила, использу-ются метрики разнообразия и новизны для уточнения позиций объектов-кандидатов в итоговом списке рекомендаций.

В рамках этой задачи выполняется подбор моделей для этапа скоринга и ранжирования.
Ограничение: использование только CPU.

### Имеющиеся данные
Социально-демографическая информация о пользователях и список их покупок - дата, название, цена, количество.
Список сгенерированных кандидатов в разрезе покупателей.

Разделение на трейн и тест: даты покупок. Самые последние покупки в тесте.
Названия датафреймов с данными: train_data, test_data, candidates.

**Состав train_data и test_data**:
```
 0   buyer_id                           object  идентификатор покупателя
 1   buyer_birth_date                   object  дата рождения покупателя
 2   buyer_sms_allowed                  int64   покупатель разрешил присылать СМС
 3   buyer_emails_allowed               int64   покупатель разрешил присылать email
 4   buyer_account_status_loyal         int64   покупатель в статусе "Лояльный" (0 или 1)
 5   buyer_account_status_unregistered  int64   покупатель в статусе "Не зарегистрирован" (0 или 1)
 6   buyer_account_status_new           int64   покупатель в статусе "Новый" (0 или 1)
 7   buyer_account_status_potential     int64   покупатель в статусе "Потенциальный" (0 или 1)
 8   buyer_account_status_lost          int64   покупатель в статусе "Потерянный" (0 или 1)
 9   buyer_account_status_sleeping      int64   покупатель в статусе "Спящий" (0 или 1)
 10  buyer_is_female                    int64   покупатель - женщина (0 или 1)
 11  buyer_age                          int64   возраст покупателя
 12  order_id                           object  идентификатор покупки
 13  order_date                         object  дата покупки
 14  order_day_of_week                  int64   день недели, в который была совершена покупка
 15  order_hour_of_day                  int64   час дня, в который была совершена покупка
 16  product_id                         object  идентификатор товара
 17  product_name                       object  название товара
 18  product_group                      object  название группы товара
 19  product_count                      float64 количество товара
 20  product_sum                        float64 сумма за товар
```

**Состав списка сгенерированных кандидатов**:
```
 0   buyer_id                           object  идентификатор покупателя
 1   product_id                         object  идентификатор товара
```


### Метрики
- NDCG@K
- Precision@K
- Recall@K
- Diversity@K
- Novelty@K
- Serendipity@K

Оптимизируем метрику NDCG@K, т.к. на этапе скоринга и ранжирования основной задачей является формирование упорядоченного по релевантности списка рекомендаций.

## Импорты и конфигурация

In [1]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Optional
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import LabelEncoder
import logging
import warnings
from multiprocessing import Pool, cpu_count
import optuna
from typing import List, Dict, Tuple, Optional
import logging
from datetime import datetime
import gc
from catboost import CatBoost, Pool as CBPool
from tqdm import tqdm
import os
import json
from optuna.trial import Trial
from lightfm import LightFM
import scipy.sparse as sparse
import xgboost as xgb
import joblib


In [2]:
ORGANIZATION_ID = ""
PROCESSING_DATE = ""
RANDOM_STATE = 42
DATA_PATH = ""

In [3]:
warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)

## Классы-утилиты и общие методы

In [4]:
class MetricsCalculator:
    """
    Калькулятор метрик для оценки качества рекомендаций.

    Args:
        k_values: список значений K для расчета метрик @K
    """

    def __init__(self, k_values: List[int]):
        self.k_values = sorted(k_values)

    @staticmethod
    def _calculate_ndcg_for_user(
        data_tuple: Tuple[str, Dict[str, np.ndarray], int],
    ) -> Optional[float]:
        """
        Вспомогательная функция для расчета NDCG одного пользователя.

        Args:
            data_tuple: кортеж (user_id, user_data, k) с данными пользователя и параметром k

        Returns:
            Optional[float]: значение NDCG для пользователя или None в случае ошибки
        """
        user_id, user_data, k = data_tuple
        try:
            return ndcg_score(
                y_true=[user_data["ideal"]], y_score=[user_data["relevance"]], k=k
            )
        except Exception as e:
            logging.error(f"Ошибка при расчете NDCG для пользователя {user_id}: {e}")
            return None

    def _calculate_ndcg_at_k(
        self,
        k: int,
        prepared_data: Dict[str, Dict[str, np.ndarray]],
    ) -> float:
        """
        Расчет метрики NDCG@K (Normalized Discounted Cumulative Gain) для заданного значения K.

        NDCG - это метрика, которая оценивает качество ранжирования рекомендаций с учетом
        их позиции в списке. В отличие от Precision и Recall, NDCG учитывает порядок
        рекомендаций, придавая больший вес релевантным товарам на верхних позициях.

        Формула расчета:
        DCG@K = rel₁ + Σᵢ₌₂ᵏ (relᵢ / log₂(i + 1))
        NDCG@K = DCG@K / IDCG@K
        где:
        - relᵢ - релевантность i-го товара (в нашем случае 0 или 1)
        - IDCG@K - максимально возможное значение DCG@K (идеальное ранжирование)

        Args:
            k: количество рекомендаций для оценки
            prepared_data: словарь подготовленных данных для расчета NDCG
                {
                    user_id: {
                        "relevance": np.ndarray,  # вектор релевантности рекомендаций
                        "ideal": np.ndarray       # вектор идеального ранжирования
                    }
                }

        Returns:
            float: среднее значение NDCG@K по всем пользователям.
            Возвращает 0.0 в следующих случаях:
            - если нет данных для расчета
            - если все попытки расчета NDCG завершились ошибкой
            - если prepared_data пуст

        Example:
            >>> prepared_data = {
            ...     'user1': {
            ...         'relevance': np.array([1, 0, 1, 0]),
            ...         'ideal': np.array([1, 1, 0, 0])
            ...     }
            ... }
            >>> calculator._calculate_ndcg_for_k(2, prepared_data)
            0.6309  # NDCG@2 для данного примера

        Notes:
            - Особенности метрики:
            * Учитывает порядок рекомендаций
            * Нормализована на [0, 1]
            * Чувствительна к позициям релевантных товаров
            * Использует логарифмическое дисконтирование

            - Интерпретация значений:
            * 1.0 - идеальное ранжирование
            * 0.0 - наихудшее возможное ранжирование
            * Типичные значения в реальных системах: 0.3-0.7

            - Важные замечания:
            * Метод использует готовые векторы релевантности
            * Требует предварительной подготовки данных
            * Обрабатывает ошибки для каждого пользователя отдельно
            * Использует реализацию из sklearn.metrics
        """
        if not prepared_data:
            return 0.0

        # Подготавливаем данные для параллельной обработки
        data_for_parallel = [
            (user_id, user_data, k) for user_id, user_data in prepared_data.items()
        ]

        # Используем контекстный менеджер для автоматического закрытия пула
        with Pool() as pool:
            # Запускаем параллельные вычисления
            results = pool.map(
                MetricsCalculator._calculate_ndcg_for_user, data_for_parallel
            )

            # Фильтруем None значения и считаем среднее
            valid_results = [r for r in results if r is not None]

            return float(np.mean(valid_results)) if valid_results else 0.0

    def _calculate_precision_for_user(
        self,
        pred_items_at_k: List[str],
        true_items: set[str],
        k: int,
    ) -> Optional[float]:
        """
        Расчет Precision@K для одного пользователя.

        Precision (точность) показывает, какая доля рекомендованных системой товаров
        в топ-K оказалась релевантной для пользователя. Эта метрика оценивает
        способность системы давать точные рекомендации без "шума".

        Формула расчета:
        Precision@K = |релевантные товары ∩ рекомендованные товары в топ-K| / K

        Args:
            pred_items_at_k: список первых K рекомендованных товаров
            true_items: множество фактически купленных товаров
            k: количество рекомендаций для оценки

        Returns:
            float: значение Precision@K для пользователя
            None: если нет данных для расчета

        Example:
            >>> pred_items_at_k = ['item1', 'item2']
            >>> true_items = {'item1', 'item4'}
            >>> calculator._calculate_precision_for_user(pred_items_at_k, true_items, k=2)
            0.5  # из двух рекомендованных товаров только один оказался релевантным

        Notes:
            - Высокое значение Precision означает, что большинство рекомендаций релевантны
            - Особенности метрики:
            * Всегда нормализована на K
            * При увеличении K обычно уменьшается
            * Не учитывает общее количество релевантных товаров
            * Чувствительна к порядку рекомендаций (в топ-K попадают первые K товаров)
        """
        if not pred_items_at_k:
            return None

        relevant_count = len(set(pred_items_at_k) & true_items)
        return relevant_count / k

    def _calculate_recall_for_user(
        self,
        pred_items_at_k: List[str],
        true_items: set[str],
    ) -> Optional[float]:
        """
        Расчет метрики Recall@K (полноты) для одного пользователя.

        Recall (полнота) показывает, какую долю от всех релевантных товаров пользователя
        система смогла рекомендовать в топ-K рекомендациях. Эта метрика оценивает
        способность системы находить все интересные пользователю товары.

        Формула расчета:
        Recall@K = |релевантные товары ∩ рекомендованные товары в топ-K| / |релевантные товары|

        Args:
            pred_items_at_k: список первых K рекомендованных товаров
            true_items: множество фактически купленных товаров

        Returns:
            float: значение Recall@K для пользователя
            None: если нет данных для расчета (пустой список рекомендаций или нет релевантных товаров)

        Example:
            >>> pred_items_at_k = ['item1', 'item2']
            >>> true_items = {'item1', 'item2', 'item4'}
            >>> calculator._calculate_recall_for_user(pred_items_at_k, true_items)
            0.667  # из трех релевантных товаров (item1, item2, item4)
                # в топ-2 рекомендациях найдено два (item1, item2)

        Notes:
            - Особенности метрики для одного пользователя:
            * Чувствительна к количеству релевантных товаров
            * При увеличении K обычно растет
            * Может быть низкой, если у пользователя много релевантных товаров
            * Не учитывает порядок рекомендаций в топ-K
            * Не учитывает нерелевантные рекомендации

            - Важные замечания:
            * Возвращает None для пустого списка рекомендаций
            * Возвращает None если нет релевантных товаров
            * Значение всегда в диапазоне [0, 1]
            * Равен 1.0, если все релевантные товары найдены
            * Равен 0.0, если не найдено ни одного релевантного товара
        """
        if not pred_items_at_k or not true_items:
            return None

        relevant_count = len(set(pred_items_at_k) & true_items)
        return relevant_count / len(true_items)

    def _calculate_diversity_for_user(
        self,
        pred_items_at_k: List[str],
        item_categories: Dict[str, str],
    ) -> Optional[float]:
        """
        Расчет метрики Diversity (разнообразия) для одного пользователя.

        Diversity (разнообразие) измеряет насколько разнообразны рекомендации с точки зрения
        категорий товаров. Высокое разнообразие означает, что система способна рекомендовать
        товары из разных категорий, а не концентрируется только на нескольких популярных
        категориях.

        Формула расчета:
        Diversity = количество уникальных категорий в рекомендациях / общее количество категорий

        Args:
            pred_items_at_k: список рекомендованных товаров
            item_categories: словарь соответствия товаров и их категорий {item_id: category_id}

        Returns:
            float: значение Diversity для пользователя
            None: если нет данных для расчета (пустой список рекомендаций или нет категорий)

        Example:
            >>> pred_items_at_k = ['item1', 'item2', 'item3']
            >>> item_categories = {
            ...     'item1': 'category1',
            ...     'item2': 'category1',
            ...     'item3': 'category2'
            ... }
            >>> calculator._calculate_diversity_for_user(pred_items_at_k, item_categories)
            0.5  # рекомендации содержат товары из 2 категорий из 4 возможных

        Notes:
            - Высокое значение разнообразия может указывать на то, что система
            предлагает пользователям широкий спектр товаров
            - Низкое значение может говорить о том, что система "зациклилась"
            на определенных категориях
            - Особенности метрики для одного пользователя:
            * Нормализована на общее количество доступных категорий
            * Не зависит от порядка рекомендаций
            * Учитывает только уникальные категории
            * Игнорирует товары без категорий

            - Важные замечания:
            * Возвращает None для пустого списка рекомендаций
            * Возвращает None если нет информации о категориях
            * Значение всегда в диапазоне [0, 1]
            * Равен 1.0, если рекомендации охватывают все категории
            * Равен 0.0, если все рекомендации из одной категории

        """
        if not pred_items_at_k or not item_categories:
            return None

        total_categories = len(set(item_categories.values()))
        if total_categories == 0:
            return None

        recommended_categories = {
            item_categories[item] for item in pred_items_at_k if item in item_categories
        }

        if not recommended_categories:
            return None

        return len(recommended_categories) / total_categories

    def _calculate_novelty_for_user(
        self,
        pred_items_at_k: List[str],
        test_items: set[str],
        k: int,
    ) -> Optional[float]:
        """
        Расчет метрики Novelty (новизны) для пользователя.

        Новизна оценивается как доля тестовых покупок пользователя,
        которые не попали в топ-K рекомендаций. Низкое значение новизны
        означает, что система хорошо предсказывает будущие покупки пользователя.

        Формула расчета:
        Novelty = |test_items - pred_items_at_k| / |test_items|

        Args:
            pred_items_at_k (List[str]):
                Список рекомендованных товаров.

            test_items (set[str]):
                Множество товаров из тестового периода пользователя.

            k (int):
                Количество рекомендаций для оценки (топ-K).

        Returns:
            Optional[float]:
                Значение новизны в диапазоне [0, 1], где:
                - 0.0 означает, что все тестовые покупки были в рекомендациях
                - 1.0 означает, что ни одна тестовая покупка не была рекомендована
                - None возвращается, если у пользователя нет тестовых покупок

        Example:
            >>> test_items = {'item1', 'item2', 'item3'}  # 3 покупки в тесте
            >>> pred_items = ['item1', 'item4', 'item2']  # top-3 рекомендации
            >>> calculator._calculate_novelty_for_user(pred_items, test_items, 3)
            0.33  # item3 не попал в рекомендации (1/3 покупок)

        Notes:
            - Особенности метрики:
            * Оценивает способность системы предсказывать будущие покупки
            * Учитывает только тестовые покупки пользователя
            * Не зависит от общего количества рекомендаций
            * Нормализована на количество тестовых покупок

            - Интерпретация значений:
            * Низкие значения - система хорошо предсказывает будущие покупки
            * Высокие значения - система не смогла предсказать многие покупки
        """
        if not test_items:
            return None

        # Берем только первые k рекомендаций
        pred_items_set = set(pred_items_at_k[:k])

        # Считаем долю тестовых покупок, которых нет в рекомендациях
        missed_items = len(test_items - pred_items_set)
        return missed_items / len(test_items)

    def _calculate_serendipity_for_user(
        self,
        pred_items_at_k: List[str],
        true_items: set[str],
        popular_items: set[str],
    ) -> Optional[float]:
        """
        Расчет метрики Serendipity (неожиданности) для одного пользователя.

        Serendipity (неожиданность) измеряет способность системы рекомендовать релевантные,
        но неочевидные товары. Товар считается неожиданным, если он релевантен для пользователя
        (присутствует в тестовых данных) и при этом не входит в множество популярных товаров.

        Эта метрика помогает оценить способность системы находить "скрытые жемчужины" -
        товары, которые нравятся пользователю, но не являются очевидным выбором.

        Формула расчета:
        Serendipity = количество релевантных неожиданных товаров / количество рекомендаций
        где релевантный неожиданный товар - это товар, который:
        1) есть в тестовых покупках пользователя (релевантный)
        2) не входит в множество популярных товаров (неожиданный)

        Args:
            pred_items_at_k: список рекомендованных товаров
            true_items: множество фактически купленных товаров из тестового периода
            popular_items: множество идентификаторов популярных товаров

        Returns:
            float: значение Serendipity для пользователя
            None: если нет данных для расчета (пустой список рекомендаций)

        Example:
            >>> pred_items_at_k = ['item1', 'item2', 'item3']
            >>> true_items = {'item1', 'item3'}
            >>> popular_items = {'item1', 'item2'}
            >>> calculator._calculate_serendipity_for_user(
            ...     pred_items_at_k, true_items, popular_items
            ... )
            0.333  # только item3 является релевантным и неожиданным
                # (item1 релевантный, но популярный; item2 популярный и нерелевантный)

        Notes:
            - Особенности метрики для одного пользователя:
            * Нормализована на количество рекомендаций
            * Не зависит от порядка рекомендаций
            * Учитывает как релевантность, так и популярность
            * Более строгая метрика, чем Novelty или Precision
            * Помогает оценить качество "неочевидных" рекомендаций

            - Важные замечания:
            * Возвращает None для пустого списка рекомендаций
            * Значение всегда в диапазоне [0, 1]
            * Равен 1.0, если все рекомендации релевантные и непопулярные
            * Равен 0.0, если нет релевантных непопулярных рекомендаций
            * Обычно имеет более низкие значения, чем другие метрики
            * Зависит от выбранного порога популярности товаров
        """
        if not pred_items_at_k:
            return None

        serendipitous_items = sum(
            1
            for item in pred_items_at_k
            if item in true_items and item not in popular_items
        )

        return serendipitous_items / len(pred_items_at_k)

    def _prepare_data(
        self,
        recommendations: Dict[str, List[str]],
        test_data: pd.DataFrame,
    ) -> Dict[str, Dict[str, np.ndarray]]:
        """
        Подготовка данных для расчета NDCG.

        Args:
            recommendations: словарь {user_id: list of recommended items}
            test_data: тестовый датафрейм с колонками 'buyer_id' и 'product_id'

        Returns:
            Dict с подготовленными данными для каждого пользователя
        """
        # Получаем фактические покупки пользователей
        true_items = test_data.groupby("buyer_id")["product_id"].agg(list).to_dict()

        # Собираем все уникальные товары
        all_items = sorted(
            list(set(item for items in recommendations.values() for item in items))
        )
        item_to_idx = {item: idx for idx, item in enumerate(all_items)}

        # Готовим структуры для расчета NDCG
        prepared_data = {}

        for user_id, pred_items in recommendations.items():
            if user_id not in true_items:
                continue

            # Создаем вектор релевантности для всех товаров
            true_set = set(true_items[user_id])
            n_items = len(all_items)

            # Создаем полный вектор релевантности
            relevance = np.zeros(n_items)
            for item in pred_items:
                if item in item_to_idx:
                    relevance[item_to_idx[item]] = 1 if item in true_set else 0

            # Создаем идеальный вектор релевантности
            ideal = np.zeros(n_items)
            for item in true_set:
                if item in item_to_idx:
                    ideal[item_to_idx[item]] = 1

            prepared_data[user_id] = {"relevance": relevance, "ideal": ideal}

        return prepared_data

    def calculate(
        self,
        recommendations: Dict[str, List[str]],
        train_data: pd.DataFrame,
        test_data: pd.DataFrame,
        item_categories: Optional[Dict[str, str]] = None,
    ) -> Dict[str, float]:
        """
        Расчет всех метрик

        Args:
            recommendations: словарь {user_id: list of recommended items}
            train_data: тренировочный датафрейм для расчета новизны
            test_data: тестовый датафрейм с колонками 'buyer_id' и 'product_id'
            item_categories: словарь {item_id: category_id} для расчета разнообразия

        Returns:
            Dict[str, float]: словарь с метриками NDCG@K, Precision@K, Recall@K,
            Diversity@K, Novelty@K и Serendipity@K для каждого значения K
        """
        # Подготавливаем общие данные
        prepared_data = self._prepare_data(recommendations, test_data)
        true_items = test_data.groupby("buyer_id")["product_id"].agg(set).to_dict()

        # Подготавливаем данные для serendipity
        item_counts = train_data["product_id"].value_counts()
        threshold = np.percentile(
            item_counts.values, 80
        )  # 1 - 0.2 (percentile по умолчанию)
        popular_items = set(item_counts[item_counts >= threshold].index)

        # Инициализируем словарь с метриками
        metrics = {}

        # Инициализируем списки для хранения значений метрик
        precision_values = []
        recall_values = []
        diversity_values = []
        novelty_values = []
        serendipity_values = []

        # Рассчитываем все метрики
        for k in self.k_values:
            metrics[f"ndcg_{k}"] = self._calculate_ndcg_at_k(k, prepared_data)

            for user_id, pred_items in recommendations.items():
                pred_items_at_k = pred_items[:k]
                user_id_in_true_items = user_id in true_items

                # Precision
                if user_id_in_true_items:
                    precision_value = self._calculate_precision_for_user(
                        pred_items_at_k=pred_items_at_k,
                        true_items=true_items[user_id],
                        k=k,
                    )
                    if precision_value is not None:
                        precision_values.append(precision_value)

                # Recall
                if user_id_in_true_items:
                    recall_value = self._calculate_recall_for_user(
                        pred_items_at_k=pred_items_at_k, true_items=true_items[user_id]
                    )
                    if recall_value is not None:
                        recall_values.append(recall_value)

                # Diversity
                diversity_value = self._calculate_diversity_for_user(
                    pred_items_at_k=pred_items_at_k,
                    item_categories=item_categories or {},
                )
                if diversity_value is not None:
                    diversity_values.append(diversity_value)

                # Novelty
                novelty_value = self._calculate_novelty_for_user(
                    pred_items_at_k=pred_items, test_items=true_items[user_id], k=k
                )
                if novelty_value is not None:
                    novelty_values.append(novelty_value)

                # Serendipity
                if user_id_in_true_items:
                    serendipity_value = self._calculate_serendipity_for_user(
                        pred_items_at_k=pred_items_at_k,
                        true_items=true_items[user_id],
                        popular_items=popular_items,
                    )
                    if serendipity_value is not None:
                        serendipity_values.append(serendipity_value)

            # Сохраняем средние значения метрик
            metrics[f"precision_{k}"] = (
                float(np.mean(precision_values)) if precision_values else 0.0
            )
            metrics[f"recall_{k}"] = (
                float(np.mean(recall_values)) if recall_values else 0.0
            )
            metrics[f"diversity_{k}"] = (
                float(np.mean(diversity_values)) if diversity_values else 0.0
            )
            metrics[f"novelty_{k}"] = (
                float(np.mean(novelty_values)) if novelty_values else 0.0
            )
            metrics[f"serendipity_{k}"] = (
                float(np.mean(serendipity_values)) if serendipity_values else 0.0
            )

        return metrics

## Бейзлайн

In [5]:
train_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv"
)
test_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv"
)
candidates = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommendations.csv"
)

# Создаем словарь категорий товаров
item_categories = {
    row["product_id"]: row["product_group"] for _, row in train_data.iterrows()
}

In [41]:
class PopularItemsRanker:
    """
    Ранжировщик на основе популярности товаров.
    """

    def __init__(self):
        self.item_scores_ = None

    def fit(
        self,
        data: pd.DataFrame,
        item_col: str = "product_id",
    ) -> "PopularItemsRanker":
        """
        Обучение ранжировщика на исторических данных.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с историческими данными о покупках
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        self : PopularItemsRanker
            Обученный объект
        """
        # Подсчет количества покупок каждого товара
        item_counts = data[item_col].value_counts()

        # Нормализация скоров
        self.item_scores_ = (item_counts / item_counts.max()).to_dict()

        return self

    def recommend(
        self,
        candidates: pd.DataFrame,
        top_n: int = 10,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
    ) -> pd.DataFrame:
        """
        Ранжирование кандидатов на основе популярности товаров.

        Args:
        ----------
        candidates : pd.DataFrame
            Датафрейм с парами пользователь-товар для ранжирования
        top_n : int, по умолчанию 10
            Количество рекомендуемых товаров для каждого пользователя
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        pd.DataFrame
            Датафрейм с ранжированными рекомендациями и их скорами
        """
        if self.item_scores_ is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit().")

        # Копируем датафрейм кандидатов
        recommendations = candidates.copy()

        # Добавляем скоры популярности
        recommendations["score"] = recommendations[item_col].map(
            lambda x: self.item_scores_.get(x, 0.0)
        )

        # Сортируем рекомендации по пользователям и скорам
        recommendations = recommendations.sort_values(
            by=[user_col, "score"], ascending=[True, False]
        )

        # Оставляем только top_n рекомендаций для каждого пользователя
        recommendations = recommendations.groupby(user_col).head(top_n)

        return recommendations

In [42]:
# Создаем и обучаем ранжировщик на тренировочных данных
logging.info("Создаем и обучаем ранжировщик")
ranker = PopularItemsRanker()
ranker.fit(train_data)

# Получаем рекомендации от ранжировщика
logging.info("Получаем рекомендации от ранжировщика")
ranked_recommendations = ranker.recommend(candidates, top_n=10)

# Преобразуем результаты ранжировщика в формат словаря для метрик
ranker_recommendations = (
    ranked_recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
)

# Создаем калькулятор метрик и считаем NDCG
metrics_calculator = MetricsCalculator(k_values=[10, 100, 1000])

logging.info("Расчет метрик для рекомендаций от PopularItemsRanker")
best_metrics = metrics_calculator.calculate(
    recommendations=ranker_recommendations,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

# Вывод результатов
logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"NDCG@{k}: {best_metrics[f'ndcg_{k}']:.4f}")
    logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")


2025-04-27 10:38:30 - INFO - Создаем и обучаем ранжировщик
2025-04-27 10:38:30 - INFO - Получаем рекомендации от ранжировщика
2025-04-27 10:39:28 - INFO - Расчет метрик для рекомендаций от PopularItemsRanker
2025-04-27 10:42:05 - INFO - Результаты:
2025-04-27 10:42:05 - INFO - Метрики для K=10:
2025-04-27 10:42:05 - INFO - NDCG@10: 0.3438
2025-04-27 10:42:05 - INFO - Precision@10: 0.0622
2025-04-27 10:42:05 - INFO - Recall@10: 0.1476
2025-04-27 10:42:05 - INFO - Diversity@10: 0.0049
2025-04-27 10:42:05 - INFO - Novelty@10: 0.8524
2025-04-27 10:42:05 - INFO - Serendipity@10: 0.0000
2025-04-27 10:42:05 - INFO - --------------------------------
2025-04-27 10:42:05 - INFO - Метрики для K=100:
2025-04-27 10:42:05 - INFO - NDCG@100: 0.4375
2025-04-27 10:42:05 - INFO - Precision@100: 0.0342
2025-04-27 10:42:05 - INFO - Recall@100: 0.1476
2025-04-27 10:42:05 - INFO - Diversity@100: 0.0049
2025-04-27 10:42:05 - INFO - Novelty@100: 0.8524
2025-04-27 10:42:05 - INFO - Serendipity@100: 0.0000
2025

## Модель 1: LightFM (Матричная факторизация)

In [5]:
train_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv"
)
test_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv"
)
candidates = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommendations.csv"
)

item_categories = {
    row["product_id"]: row["product_group"] for _, row in train_data.iterrows()
}

In [6]:
class MatrixFactorizationRanker:
    """
    Ранжировщик на основе матричной факторизации с использованием LightFM.

    Attrs:
    ----------
    model : LightFM
        Модель матричной факторизации
    user_encoder : LabelEncoder
        Энкодер для преобразования ID пользователей в индексы
    item_encoder : LabelEncoder
        Энкодер для преобразования ID товаров в индексы
    user_features_matrix : sparse.csr_matrix
        Разреженная матрица признаков пользователей
    n_users : int
        Количество пользователей
    n_items : int
        Количество товаров
    """

    def __init__(
        self,
        n_components: int = 64,
        learning_rate: float = 0.05,
        loss: str = "warp",
        random_state: int = 42,
        n_epochs: int = 30,
    ):
        """
        Инициализация ранжировщика.

        Args:
        ----------
        n_components : int, по умолчанию 64
            Размерность латентных векторов
        learning_rate : float, по умолчанию 0.05
            Скорость обучения
        loss : str, по умолчанию 'warp'
            Функция потерь ('warp', 'bpr', 'warp-kos')
        random_state : int, по умолчанию 42
            Seed для воспроизводимости результатов
        n_epochs : int, по умолчанию 30
            Количество эпох обучения
        """
        self.model = LightFM(
            no_components=n_components,
            learning_rate=learning_rate,
            loss=loss,
            random_state=random_state,
        )
        self.n_epochs = n_epochs
        self.user_encoder = LabelEncoder()
        self.item_encoder = LabelEncoder()
        self.user_features_matrix = None
        self.n_users = None
        self.n_items = None

    def _prepare_user_features(
        self,
        data: pd.DataFrame,
        user_col: str = "buyer_id",
    ) -> sparse.csr_matrix:
        """
        Подготовка признаков пользователей.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с данными пользователей
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей

        Returns:
        ----------
        sparse.csr_matrix
            Разреженная матрица признаков пользователей
        """
        feature_columns = [
            "buyer_sms_allowed",
            "buyer_emails_allowed",
            "buyer_account_status_loyal",
            "buyer_account_status_unregistered",
            "buyer_account_status_new",
            "buyer_account_status_potential",
            "buyer_account_status_lost",
            "buyer_account_status_sleeping",
            "buyer_is_female",
            "buyer_age",
        ]

        # Получаем уникальные значения для каждого пользователя
        user_features = data.groupby(user_col)[feature_columns].first().reset_index()
        user_features["user_idx"] = self.user_encoder.transform(user_features[user_col])

        # Создаем массивы для разреженной матрицы
        rows = []
        cols = []
        data_values = []

        # Для каждого признака создаем соответствующие элементы разреженной матрицы
        for feature_idx, feature_name in enumerate(feature_columns):
            feature_values = user_features[feature_name].values
            # Для числовых признаков (например, возраста) используем значение напрямую
            if feature_name == "buyer_age":
                # Нормализуем возраст и обрабатываем пропущенные значения
                valid_mask = ~np.isnan(feature_values)
                rows.extend(user_features.loc[valid_mask, "user_idx"])
                cols.extend([feature_idx] * np.sum(valid_mask))
                data_values.extend(feature_values[valid_mask])
            else:
                # Для бинарных признаков используем 1 только если значение True
                valid_mask = feature_values == True
                rows.extend(user_features.loc[valid_mask, "user_idx"])
                cols.extend([feature_idx] * np.sum(valid_mask))
                data_values.extend([1.0] * np.sum(valid_mask))

        # Создаем разреженную матрицу
        return sparse.csr_matrix(
            (data_values, (rows, cols)), shape=(self.n_users, len(feature_columns))
        )

    def _prepare_interaction_matrix(
        self,
        data: pd.DataFrame,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
    ) -> sparse.coo_matrix:
        """
        Подготовка матрицы взаимодействий пользователь-товар.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с данными о взаимодействиях
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        sparse.coo_matrix
            Разреженная матрица взаимодействий
        """
        interactions = (
            data.groupby([user_col, item_col]).size().reset_index(name="count")
        )

        interactions["user_idx"] = self.user_encoder.transform(interactions[user_col])
        interactions["item_idx"] = self.item_encoder.transform(interactions[item_col])

        return sparse.coo_matrix(
            (
                interactions["count"],
                (interactions["user_idx"], interactions["item_idx"]),
            ),
            shape=(self.n_users, self.n_items),
        )

    def fit(
        self,
        data: pd.DataFrame,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
    ) -> "MatrixFactorizationRanker":
        """
        Обучение ранжировщика на исторических данных.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с историческими данными о покупках
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        self : MatrixFactorizationRanker
            Обученный объект
        """
        # Кодируем ID пользователей и товаров
        self.user_encoder.fit(data[user_col].unique())
        self.item_encoder.fit(data[item_col].unique())

        self.n_users = len(self.user_encoder.classes_)
        self.n_items = len(self.item_encoder.classes_)

        # Подготавливаем признаки и матрицу взаимодействий
        self.user_features_matrix = self._prepare_user_features(data, user_col)
        interaction_matrix = self._prepare_interaction_matrix(data, user_col, item_col)

        num_threads = cpu_count()
        # Обучаем модель
        self.model.fit(
            interaction_matrix,
            user_features=self.user_features_matrix,
            epochs=self.n_epochs,
            verbose=True,
            num_threads=num_threads,
        )

        return self

    def rank(
        self,
        candidates: pd.DataFrame,
        top_n: int = 10,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
    ) -> pd.DataFrame:
        """
        Ранжирование кандидатов на основе матричной факторизации.

        Args:
        ----------
        candidates : pd.DataFrame
            Датафрейм с парами пользователь-товар для ранжирования
        top_n : int, по умолчанию 10
            Количество рекомендуемых товаров для каждого пользователя
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        pd.DataFrame
            Датафрейм с ранжированными рекомендациями и их скорами
        """
        if not hasattr(self, "model") or self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit().")

        recommendations = candidates.copy()

        # Фильтруем только известные пользователи и товары
        known_users_mask = recommendations[user_col].isin(self.user_encoder.classes_)
        known_items_mask = recommendations[item_col].isin(self.item_encoder.classes_)

        valid_recommendations = recommendations[
            known_users_mask & known_items_mask
        ].copy()

        if valid_recommendations.empty:
            return pd.DataFrame(columns=recommendations.columns + ["score"])

        # Преобразуем ID в индексы только для известных пользователей и товаров
        valid_recommendations["user_idx"] = self.user_encoder.transform(
            valid_recommendations[user_col]
        )
        valid_recommendations["item_idx"] = self.item_encoder.transform(
            valid_recommendations[item_col]
        )

        num_threads = cpu_count()
        # Получаем предсказания модели для всех пар сразу
        scores = self.model.predict(
            user_ids=valid_recommendations["user_idx"].values,
            item_ids=valid_recommendations["item_idx"].values,
            user_features=self.user_features_matrix,
            num_threads=num_threads,
        )

        valid_recommendations["score"] = scores

        # Сортируем рекомендации
        valid_recommendations = valid_recommendations.sort_values(
            by=[user_col, "score"], ascending=[True, False]
        )

        # Оставляем top_n рекомендаций для каждого пользователя
        valid_recommendations = valid_recommendations.groupby(user_col).head(top_n)

        # Удаляем временные столбцы
        valid_recommendations = valid_recommendations.drop(
            ["user_idx", "item_idx"], axis=1
        )

        return valid_recommendations

    def save_model(self, model_path: str, export_parameters: bool = True) -> None:
        """
        Сохранение модели и её параметров.

        Args:
            model_path: путь для сохранения модели
            export_parameters: сохранять ли дополнительные параметры модели
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit()")

        # Создаем директорию если её нет
        os.makedirs(os.path.dirname(model_path), exist_ok=True)

        # Сохраняем модель LightFM
        joblib.dump(self.model, model_path)

        if export_parameters:
            # Сохраняем дополнительные параметры модели
            parameters = {
                "user_encoder": self.user_encoder,
                "item_encoder": self.item_encoder,
                "user_features_matrix": self.user_features_matrix,
                "n_users": self.n_users,
                "n_items": self.n_items,
                "n_epochs": self.n_epochs,
            }

            # Путь для сохранения параметров
            params_path = f"{os.path.splitext(model_path)[0]}_params.joblib"

            joblib.dump(parameters, params_path)

    @classmethod
    def load_model(
        cls, model_path: str, load_parameters: bool = True
    ) -> "MatrixFactorizationRanker":
        """
        Загрузка сохранённой модели и её параметров.

        Args:
            model_path: путь к сохранённой модели
            load_parameters: загружать ли дополнительные параметры модели

        Returns:
            MatrixFactorizationRanker: загруженная модель
        """
        # Создаем экземпляр класса
        ranker = cls()

        # Загружаем модель LightFM
        ranker.model = joblib.load(model_path)

        if load_parameters:
            # Путь к файлу с параметрами
            params_path = f"{os.path.splitext(model_path)[0]}_params.joblib"

            if os.path.exists(params_path):
                parameters = joblib.load(params_path)

                # Загружаем параметры
                ranker.user_encoder = parameters["user_encoder"]
                ranker.item_encoder = parameters["item_encoder"]
                ranker.user_features_matrix = parameters["user_features_matrix"]
                ranker.n_users = parameters["n_users"]
                ranker.n_items = parameters["n_items"]
                ranker.n_epochs = parameters["n_epochs"]
            else:
                logging.warning(
                    f"Файл с параметрами {params_path} не найден. "
                    "Загружена только модель без дополнительных параметров."
                )

        return ranker

In [26]:
def optimize_lightfm_parameters(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    candidates: pd.DataFrame,
    metrics_calculator: MetricsCalculator,
    n_trials: int = 10,
    timeout: int = 7200,
    item_categories: Dict[str, str] = None,
) -> Tuple[Dict, Dict[str, float]]:
    """
    Оптимизация параметров MatrixFactorizationRanker с помощью Optuna.

    Args:
        train_data: тренировочные данные
        test_data: тестовые данные
        candidates: кандидаты для ранжирования
        metrics_calculator: калькулятор метрик
        n_trials: количество итераций оптимизации
        timeout: максимальное время оптимизации в секундах

    Returns:
        Tuple[Dict, Dict[str, float]]:
            - Лучшие параметры
            - Значения метрик на лучших параметрах
    """

    def objective(trial: Trial) -> float:
        # Определяем пространство гиперпараметров
        model_params = {
            "n_components": trial.suggest_int("n_components", 32, 256),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.5, log=True),
            "loss": trial.suggest_categorical("loss", ["warp", "bpr", "warp-kos"]),
            "n_epochs": trial.suggest_int("n_epochs", 10, 50),
            "random_state": 42,
        }

        # Создаем и обучаем модель
        ranker = MatrixFactorizationRanker(
            n_components=model_params["n_components"],
            learning_rate=model_params["learning_rate"],
            loss=model_params["loss"],
            random_state=model_params["random_state"],
            n_epochs=model_params["n_epochs"],
        )

        # Обучаем модель
        ranker.fit(train_data)

        # Получаем предсказания
        recommendations = ranker.rank(
            candidates=candidates, top_n=max(metrics_calculator.k_values)
        )

        if recommendations.empty:
            return 0.0

        # Преобразуем в формат для расчета метрик
        recommendations_dict = (
            recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
        )

        # Рассчитываем метрики
        metrics = metrics_calculator.calculate(
            recommendations=recommendations_dict,
            train_data=train_data,
            test_data=test_data,
        )

        # Возвращаем метрику для оптимизации (NDCG@K)
        target_k = max(metrics_calculator.k_values)
        return metrics.get(f"ndcg_{target_k}", 0.0)

    # Создаем исследование
    study = optuna.create_study(
        direction="maximize",
        study_name=f'lightfm_ranker_optimization_{datetime.now().strftime("%Y%m%d_%H%M%S")}',
    )

    # Запускаем оптимизацию
    logging.info("Начало оптимизации параметров...")
    study.optimize(
        objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True
    )

    # Получаем лучшие параметры
    best_params = study.best_params

    # Добавляем фиксированные параметры
    best_model_params = {
        "n_components": best_params["n_components"],
        "learning_rate": best_params["learning_rate"],
        "loss": best_params["loss"],
        "n_epochs": best_params["n_epochs"],
        "random_state": 42,
    }

    # Обучаем модель на лучших параметрах
    logging.info("Обучение модели на лучших параметрах...")
    best_ranker = MatrixFactorizationRanker(**best_model_params)
    best_ranker.fit(train_data)

    # Получаем предсказания
    recommendations = best_ranker.rank(
        candidates=candidates, top_n=max(metrics_calculator.k_values)
    )

    # Рассчитываем финальные метрики
    if recommendations.empty:
        final_metrics = {f"ndcg_{k}": 0.0 for k in metrics_calculator.k_values}
    else:
        recommendations_dict = (
            recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
        )

        final_metrics = metrics_calculator.calculate(
            recommendations=recommendations_dict,
            train_data=train_data,
            test_data=test_data,
            item_categories=item_categories
        )

    return best_params, final_metrics

In [28]:
if __name__ == "__main__":
    n_samples = 1000
    # buyer_ids = test_data["buyer_id"].unique()[:n_samples]
    buyer_ids = test_data["buyer_id"].unique()

    temp_candidates = candidates[candidates["buyer_id"].isin(buyer_ids)].copy()

    product_ids = temp_candidates["product_id"].unique()
    temp_train_data = train_data[
        (train_data["buyer_id"].isin(buyer_ids))
        & (train_data["product_id"].isin(product_ids))
    ].copy()

    temp_test_data = test_data[
        (test_data["buyer_id"].isin(buyer_ids))
        & (test_data["product_id"].isin(product_ids))
    ].copy()

    metrics_calculator = MetricsCalculator([10, 100, 1000])

    best_params, best_metrics = optimize_lightfm_parameters(
        train_data=temp_train_data,
        test_data=temp_test_data,
        candidates=temp_candidates,
        metrics_calculator=metrics_calculator,
        n_trials=10,
        timeout=7200,
        item_categories=item_categories,
    )

    logging.info(f"Лучшие параметры: {best_params}")
    logging.info("Результаты:")
    for k in metrics_calculator.k_values:
        logging.info(f"Метрики для K={k}:")
        logging.info(f"NDCG@{k}: {best_metrics[f'ndcg_{k}']:.4f}")
        logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
        logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
        logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
        logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
        logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
        logging.info("--------------------------------")

[I 2025-04-25 16:11:35,544] A new study created in memory with name: lightfm_ranker_optimization_20250425_161135
2025-04-25 16:11:35 - INFO - Начало оптимизации параметров...
Best trial: 0. Best value: 0.832345:  10%|█         | 1/10 [12:14<1:50:07, 734.17s/it, 734.17/7200 seconds]

[I 2025-04-25 16:23:49,714] Trial 0 finished with value: 0.8323453801529145 and parameters: {'n_components': 184, 'learning_rate': 0.36596795884334166, 'loss': 'warp-kos', 'n_epochs': 23}. Best is trial 0 with value: 0.8323453801529145.


Epoch: 100%|██████████| 20/20 [03:11<00:00,  9.56s/it]


[I 2025-04-25 16:34:37,780] Trial 1 finished with value: 0.8323453801529145 and parameters: {'n_components': 247, 'learning_rate': 0.0058284023495519775, 'loss': 'warp', 'n_epochs': 20}. Best is trial 0 with value: 0.8323453801529145.


Best trial: 0. Best value: 0.832345:  20%|██        | 2/10 [31:50<1:31:08, 683.52s/it, 1382.24/7200 seconds]

...

Best trial: 0. Best value: 0.832345:  90%|█████████ | 9/10 [1:37:25<10:12, 612.13s/it, 5845.05/7200 seconds]

[I 2025-04-25 17:49:00,598] Trial 8 finished with value: 0.8323453801529145 and parameters: {'n_components': 162, 'learning_rate': 0.004469104521297052, 'loss': 'bpr', 'n_epochs': 27}. Best is trial 0 with value: 0.8323453801529145.


Epoch: 100%|██████████| 25/25 [02:27<00:00,  5.88s/it]


[I 2025-04-25 17:58:18,650] Trial 9 finished with value: 0.8323453801529145 and parameters: {'n_components': 145, 'learning_rate': 0.0014783076507807222, 'loss': 'warp', 'n_epochs': 25}. Best is trial 0 with value: 0.8323453801529145.


Best trial: 0. Best value: 0.832345: 100%|██████████| 10/10 [1:46:43<00:00, 640.31s/it, 6403.11/7200 seconds]
2025-04-25 17:58:18 - INFO - Обучение модели на лучших параметрах...
Epoch: 100%|██████████| 23/23 [04:49<00:00, 12.59s/it]
2025-04-25 18:12:26 - INFO - Лучшие параметры: {'n_components': 184, 'learning_rate': 0.36596795884334166, 'loss': 'warp-kos', 'n_epochs': 23}
2025-04-25 18:12:26 - INFO - Результаты:
2025-04-25 18:12:26 - INFO - Метрики для K=10:
2025-04-25 18:12:26 - INFO - NDCG@10: 0.8398
2025-04-25 18:12:26 - INFO - Precision@10: 0.0019
2025-04-25 18:12:26 - INFO - Recall@10: 0.0035
2025-04-25 18:12:26 - INFO - Diversity@10: 0.0056
2025-04-25 18:12:26 - INFO - Novelty@10: 0.9948
2025-04-25 18:12:26 - INFO - Serendipity@10: 0.0001
2025-04-25 18:12:26 - INFO - --------------------------------
2025-04-25 18:12:26 - INFO - Метрики для K=100:
2025-04-25 18:12:26 - INFO - NDCG@100: 0.8277
2025-04-25 18:12:26 - INFO - Precision@100: 0.0029
2025-04-25 18:12:26 - INFO - Recall@

## Модель 2: CatBoost (Градиентный бустинг над решающими деревьями)

In [ ]:
train_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv"
)
test_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv"
)
candidates = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommendations.csv"
)

item_categories = {
    row["product_id"]: row["product_group"] for _, row in train_data.iterrows()
}

In [9]:
class CatBoostRanker:
    """
    Ранжировщик на основе CatBoost.

    Attrs:
        model: CatBoost
            Модель CatBoost для ранжирования
        feature_names: List[str]
            Список используемых признаков
        categorical_features: List[str]
            Список категориальных признаков
        numeric_fill_values_: Dict[str, float]
            Словарь для хранения значений для заполнения пропусков в числовых признаках
    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        iterations: int = 1000,
        depth: int = 6,
        l2_leaf_reg: float = 3.0,
        random_seed: int = 42,
        thread_count: int = -1,
        verbose: bool = True,
    ):
        """
        Инициализация ранжировщика.

        Args:
            learning_rate: скорость обучения
            iterations: количество итераций
            depth: глубина деревьев
            l2_leaf_reg: L2 регуляризация
            random_seed: seed для воспроизводимости
            thread_count: количество потоков (-1 для использования всех)
            verbose: выводить ли прогресс обучения
        """
        self.model_params = {
            "learning_rate": learning_rate,
            "iterations": iterations,
            "depth": depth,
            "l2_leaf_reg": l2_leaf_reg,
            "random_seed": random_seed,
            "thread_count": thread_count,
            "verbose": verbose,
            "task_type": "CPU",
            "loss_function": "Logloss",
            "eval_metric": "NDCG",
            "early_stopping_rounds": 50,
        }

        self.feature_names = [
            # Социально-демографические признаки
            "buyer_sms_allowed",
            "buyer_emails_allowed",
            "buyer_account_status_loyal",
            "buyer_account_status_unregistered",
            "buyer_account_status_new",
            "buyer_account_status_potential",
            "buyer_account_status_lost",
            "buyer_account_status_sleeping",
            "buyer_is_female",
            "buyer_age",
            # Временные признаки
            "order_day_of_week",
            "order_hour_of_day",
            # Признаки покупок
            "product_count",
            "product_sum",
            # Категориальные признаки
            "product_group",
        ]

        self.categorical_features = ["product_group"]
        self.model = None
        self.numeric_fill_values_ = {}  # Добавляем инициализацию словаря

    def _prepare_features(
        self, data: pd.DataFrame, is_train: bool = True
    ) -> Tuple[pd.DataFrame, Optional[np.ndarray]]:
        """
        Подготовка признаков для обучения или предсказания.

        Args:
            data: исходный датафрейм
            is_train: флаг обучения/предсказания

        Returns:
            Tuple с подготовленными признаками и целевой переменной (если is_train=True)
        """
        features = data[self.feature_names].copy()

        # Заполняем пропуски
        for col in features.columns:
            if col in self.categorical_features:
                features[col] = features[col].fillna("unknown")
            else:
                # Для числовых признаков используем 0 или медиану
                if is_train:
                    fill_value = (
                        features[col].median() if features[col].notnull().any() else 0
                    )
                    self.numeric_fill_values_ = self.numeric_fill_values_ or {}
                    self.numeric_fill_values_[col] = fill_value
                else:
                    fill_value = self.numeric_fill_values_.get(col, 0)
                features[col] = features[col].fillna(fill_value)

        if is_train:
            # Для обучения создаем целевую переменную (1 для реальных покупок)
            target = np.ones(len(features))
            return features, target
        else:
            return features, None

    def fit(
        self,
        train_data: pd.DataFrame,
        candidates: pd.DataFrame,
        validation_size: float = 0.2,
    ) -> "CatBoostRanker":
        """
        Обучение модели ранжирования.

        Args:
            train_data: исторические данные о покупках
            candidates: датафрейм с кандидатами
            validation_size: доля данных для валидации

        Returns:
            self: обученная модель
        """
        # Сортируем данные по buyer_id
        train_data = train_data.sort_values("buyer_id")
        candidates = candidates.sort_values("buyer_id")

        # Подготовка положительных примеров
        X_pos, y_pos = self._prepare_features(train_data, is_train=True)
        groups_pos = train_data["buyer_id"].values

        # Подготовка отрицательных примеров (кандидаты)
        features_df = train_data[["buyer_id", "product_id"] + self.feature_names].copy()
        features_df = features_df.drop_duplicates(subset=["buyer_id", "product_id"])

        del train_data
        gc.collect()

        chunk_size = 100000
        X_chunks = []
        groups_chunks = []
        total_negative_samples = 0

        for start_idx in tqdm(
            range(0, len(candidates), chunk_size),
            desc="Подготовка отрицательных примеров",
        ):
            end_idx = start_idx + chunk_size
            chunk = candidates.iloc[start_idx:end_idx]

            chunk_with_features = chunk.merge(
                features_df, on=["buyer_id", "product_id"], how="left"
            )

            X_chunk, _ = self._prepare_features(chunk_with_features, is_train=True)
            groups_chunk = chunk_with_features["buyer_id"].values

            # Добавляем подготовленные данные в списки
            X_chunks.append(X_chunk)
            groups_chunks.append(groups_chunk)
            total_negative_samples += len(X_chunk)

            del chunk_with_features, X_chunk, groups_chunk
            gc.collect()

        del features_df
        gc.collect()

        # Создаем массивы для отрицательных примеров
        X_neg = pd.DataFrame(columns=X_pos.columns)
        groups_neg = np.empty(total_negative_samples, dtype=groups_pos.dtype)
        y_neg = np.zeros(total_negative_samples)

        # Заполняем массивы порциями
        current_idx = 0
        for X_chunk, groups_chunk in zip(X_chunks, groups_chunks):
            chunk_size = len(X_chunk)
            X_neg = pd.concat([X_neg, X_chunk], ignore_index=True)
            groups_neg[current_idx : current_idx + chunk_size] = groups_chunk
            current_idx += chunk_size

            del X_chunk
            gc.collect()

        del X_chunks, groups_chunks
        gc.collect()

        # Объединяем положительные и отрицательные примеры
        X = pd.concat([X_pos, X_neg], ignore_index=True)
        y = np.concatenate([y_pos, y_neg])
        groups = np.concatenate([groups_pos, groups_neg])

        del X_pos, X_neg, y_pos, y_neg, groups_pos, groups_neg
        gc.collect()

        # Сортируем все данные по группам
        sort_idx = np.argsort(groups)
        X = X.iloc[sort_idx].reset_index(drop=True)
        y = y[sort_idx]
        groups = groups[sort_idx]

        # Разделение на обучающую и валидационную выборки
        unique_groups = np.unique(groups)
        n_groups = len(unique_groups)
        np.random.shuffle(unique_groups)
        train_groups = set(unique_groups[: int(n_groups * (1 - validation_size))])

        train_mask = np.array([g in train_groups for g in groups])
        valid_mask = ~train_mask

        train_pool = CBPool(
            data=X[train_mask],
            label=y[train_mask],
            group_id=groups[train_mask],
            cat_features=self.categorical_features,
        )

        valid_pool = CBPool(
            data=X[valid_mask],
            label=y[valid_mask],
            group_id=groups[valid_mask],
            cat_features=self.categorical_features,
        )

        del X, y, groups, train_mask, valid_mask
        gc.collect()

        self.model = CatBoost(self.model_params)
        self.model.fit(train_pool, eval_set=valid_pool, plot=False)

        return self

    def rank(
        self, candidates: pd.DataFrame, train_data: pd.DataFrame, top_n: int = 10
    ) -> pd.DataFrame:
        """
        Ранжирование кандидатов.

        Args:
            candidates: датафрейм с кандидатами
            train_data: исторические данные для получения признаков
            top_n: количество рекомендаций для каждого пользователя

        Returns:
            DataFrame с ранжированными рекомендациями
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit().")

        # Сортируем кандидатов по buyer_id
        candidates = candidates.sort_values("buyer_id").reset_index(drop=True)

        # Берем только нужные колонки из train_data для уменьшения размера при merge
        features_df = train_data[
            ["buyer_id", "product_id"] + self.feature_names
        ].drop_duplicates()

        # Объединяем кандидатов с историческими данными
        candidates_with_features = candidates.merge(
            features_df, on=["buyer_id", "product_id"], how="left"
        )

        # Проверяем, что количество строк не изменилось
        if len(candidates_with_features) != len(candidates):
            # Убираем возможные дубликаты
            candidates_with_features = candidates_with_features.drop_duplicates(
                subset=["buyer_id", "product_id"]
            ).reset_index(drop=True)

        # Подготавливаем признаки
        X_pred, _ = self._prepare_features(candidates_with_features, is_train=False)

        # Получаем предсказания
        scores = self.model.predict(
            CBPool(data=X_pred, cat_features=self.categorical_features)
        )

        # Проверяем размерности
        if len(scores) != len(candidates):
            raise ValueError(
                f"Количество предсказаний ({len(scores)}) не совпадает с "
                f"количеством кандидатов ({len(candidates)})"
            )

        # Формируем рекомендации
        recommendations = candidates.copy()
        recommendations["score"] = scores

        # Сортируем и отбираем top_n рекомендаций для каждого пользователя
        recommendations = recommendations.sort_values(
            by=["buyer_id", "score"], ascending=[True, False]
        )
        recommendations = recommendations.groupby("buyer_id").head(top_n)

        return recommendations[["buyer_id", "product_id", "score"]]

    def save_model(
        self, model_path: str, format: str = "cbm", export_parameters: bool = True
    ) -> None:
        """
        Сохранение модели и её параметров.

        Args:
            model_path: путь для сохранения модели
            format: формат сохранения ('cbm' или 'json')
            export_parameters: сохранять ли дополнительные параметры модели
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit()")

        # Создаем директорию если её нет
        os.makedirs(os.path.dirname(model_path), exist_ok=True)

        # Сохраняем модель CatBoost
        self.model.save_model(model_path, format=format, pool=None)

        if export_parameters:
            # Сохраняем дополнительные параметры модели
            parameters = {
                "model_params": self.model_params,
                "feature_names": self.feature_names,
                "categorical_features": self.categorical_features,
                "numeric_fill_values": self.numeric_fill_values_,
            }

            # Путь для сохранения параметров
            params_path = f"{os.path.splitext(model_path)[0]}_params.json"

            with open(params_path, "w", encoding="utf-8") as f:
                json.dump(parameters, f, ensure_ascii=False, indent=2)

    @classmethod
    def load_model(
        cls, model_path: str, load_parameters: bool = True
    ) -> "CatBoostRanker":
        """
        Загрузка сохранённой модели и её параметров.

        Args:
            model_path: путь к сохранённой модели
            load_parameters: загружать ли дополнительные параметры модели

        Returns:
            CatBoostRanker: загруженная модель
        """
        # Создаем экземпляр класса
        ranker = cls()

        # Загружаем модель CatBoost
        ranker.model = CatBoost()
        ranker.model.load_model(model_path)

        if load_parameters:
            # Путь к файлу с параметрами
            params_path = f"{os.path.splitext(model_path)[0]}_params.json"

            if os.path.exists(params_path):
                with open(params_path, "r", encoding="utf-8") as f:
                    parameters = json.load(f)

                # Загружаем параметры
                ranker.model_params = parameters["model_params"]
                ranker.feature_names = parameters["feature_names"]
                ranker.categorical_features = parameters["categorical_features"]
                ranker.numeric_fill_values_ = parameters["numeric_fill_values"]
            else:
                logging.warning(
                    f"Файл с параметрами {params_path} не найден. "
                    "Загружена только модель без дополнительных параметров."
                )

        return ranker

In [ ]:
def optimize_catboost_parameters(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    candidates: pd.DataFrame,
    metrics_calculator: MetricsCalculator,
    n_trials: int = 10,
    timeout: int = 7200,
) -> Tuple[Dict, Dict[str, float]]:
    """
    Оптимизация параметров CatBoostRanker с помощью Optuna.

    Args:
        train_data: тренировочные данные
        test_data: тестовые данные
        candidates: кандидаты для ранжирования
        n_trials: количество итераций оптимизации
        timeout: максимальное время оптимизации в секундах

    Returns:
        Tuple[Dict, Dict[str, float]]:
            - Лучшие параметры
            - Значения метрик на лучших параметрах
    """

    def objective(trial: Trial) -> float:
        # Определяем пространство гиперпараметров
        model_params = {
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
            'iterations': 1000,
            'depth': trial.suggest_int('depth', 4, 10),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
            'random_strength': trial.suggest_float('random_strength', 1e-8, 10.0, log=True),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
            'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 10),
            'early_stopping_rounds': 50,
            'task_type': 'CPU',
            'thread_count': -1,
            'verbose': False,
            'eval_metric': 'NDCG',
            'loss_function': 'Logloss'
        }

        # Создаем и обучаем модель
        ranker = CatBoostRanker(
            learning_rate=model_params['learning_rate'],
            iterations=model_params['iterations'],
            depth=model_params['depth'],
            l2_leaf_reg=model_params['l2_leaf_reg'],
            random_seed=42,
            thread_count=model_params['thread_count'],
            verbose=model_params['verbose']
        )

        # Обновляем model_params в объекте
        ranker.model_params.update(model_params)

        # Обучаем модель
        ranker.fit(train_data, candidates)

        # Получаем предсказания
        recommendations = ranker.rank(
            candidates=candidates,
            train_data=train_data,
            top_n=max([10, 100, 1000]),  # для расчета всех метрик
        )

        # Преобразуем в формат для расчета метрик
        recommendations_dict = (
            recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
        )

        # Рассчитываем метрики
        metrics = metrics_calculator.calculate(
            recommendations=recommendations_dict,
            train_data=train_data,
            test_data=test_data,
            item_categories=item_categories,
        )

        # Возвращаем метрику для оптимизации (NDCG@K)
        target_k = max(metrics_calculator.k_values)
        return metrics[f"ndcg_{target_k}"]

    # Создаем исследование
    study = optuna.create_study(
        direction="maximize",
        study_name=f'catboost_ranker_optimization_{datetime.now().strftime("%Y%m%d_%H%M%S")}',
    )

    # Запускаем оптимизацию
    logging.info("Начало оптимизации параметров...")
    study.optimize(
        objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True
    )

    # Получаем лучшие параметры
    best_params = study.best_params

    # Добавляем фиксированные параметры
    best_model_params = {
        'learning_rate': best_params['learning_rate'],
        'iterations': 1000,
        'depth': best_params['depth'],
        'l2_leaf_reg': best_params['l2_leaf_reg'],
        'random_seed': 42,
        'thread_count': -1,
        'verbose': True,
        'task_type': 'CPU',
        'loss_function': 'Logloss',
        'eval_metric': 'NDCG',
        'early_stopping_rounds': 50,
        'random_strength': best_params['random_strength'],
        'bagging_temperature': best_params['bagging_temperature'],
        'leaf_estimation_iterations': best_params['leaf_estimation_iterations']
    }

    # Обучаем модель на лучших параметрах
    logging.info("Обучение модели на лучших параметрах...")
    best_ranker = CatBoostRanker(
        learning_rate=best_model_params['learning_rate'],
        iterations=best_model_params['iterations'],
        depth=best_model_params['depth'],
        l2_leaf_reg=best_model_params['l2_leaf_reg'],
        random_seed=42,
        thread_count=best_model_params['thread_count'],
        verbose=best_model_params['verbose']
    )
    best_ranker.model_params.update(best_model_params)
    best_ranker.fit(train_data, candidates)

    # Получаем предсказания
    recommendations = best_ranker.rank(
        candidates=candidates, train_data=train_data, top_n=max([10, 100, 1000])
    )

    # Рассчитываем финальные метрики
    recommendations_dict = (
        recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
    )

    final_metrics = metrics_calculator.calculate(
        recommendations=recommendations_dict,
        train_data=train_data,
        test_data=test_data,
        item_categories=item_categories,
    )

    # Выводим результаты
    logging.info("\nЛучшие параметры:")
    for param_name, param_value in best_params.items():
        logging.info(f"{param_name}: {param_value}")

    logging.info("\nМетрики на лучших параметрах:")
    for metric_name, metric_value in final_metrics.items():
        logging.info(f"{metric_name}: {metric_value:.4f}")

    return best_params, final_metrics

In [ ]:
if __name__ == "__main__":
    n_samples = 100
    # buyer_ids = test_data["buyer_id"].unique()[:n_samples]
    buyer_ids = test_data["buyer_id"].unique()

    temp_candidates = candidates[candidates["buyer_id"].isin(buyer_ids)].copy()

    product_ids = temp_candidates["product_id"].unique()
    temp_train_data = train_data[
        (train_data["buyer_id"].isin(buyer_ids))
        & (train_data["product_id"].isin(product_ids))
    ].copy()

    temp_test_data = test_data[
        (test_data["buyer_id"].isin(buyer_ids))
        & (test_data["product_id"].isin(product_ids))
    ].copy()

    metrics_calculator = MetricsCalculator([10, 100, 1000])

    best_params, best_metrics = optimize_catboost_parameters(
        train_data=temp_train_data,
        test_data=temp_test_data,
        candidates=temp_candidates,
        metrics_calculator=metrics_calculator,
        n_trials=10,
        timeout=7200,
    )

    logging.info(f"Лучшие параметры: {best_params}")
    logging.info("Результаты:")
    for k in metrics_calculator.k_values:
        logging.info(f"Метрики для K={k}:")
        logging.info(f"NDCG@{k}: {best_metrics[f'ndcg_{k}']:.4f}")
        logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
        logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
        logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
        logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
        logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
        logging.info("--------------------------------")

[I 2025-04-21 12:00:34,560] A new study created in memory with name: catboost_ranker_optimization_20250421_120034
2025-04-21 12:00:34 - INFO - Начало оптимизации параметров...
Подготовка отрицательных примеров: 100%|██████████| 601/601 [05:16<00:00,  1.90it/s]
IOStream.flush timed out
  0%|          | 0/10 [1:28:41<?, ?it/s]

[I 2025-04-21 13:29:15,605] Trial 0 finished with value: 0.8317919801478025 and parameters: {'learning_rate': 0.004684283500572371, 'depth': 6, 'l2_leaf_reg': 5.36104374072008e-06, 'random_strength': 0.007652648646042903, 'bagging_temperature': 0.9487648394691982, 'leaf_estimation_iterations': 9}. Best is trial 0 with value: 0.8317919801478025.


Best trial: 0. Best value: 0.831792:  10%|█         | 1/10 [2:37:40<13:18:09, 5321.05s/it, 5321.05/7200 seconds]

[I 2025-04-21 14:38:15,522] Trial 1 finished with value: 0.8317919801478025 and parameters: {'learning_rate': 0.02388179353529231, 'depth': 5, 'l2_leaf_reg': 4.8339398459122034e-08, 'random_strength': 0.0031822125482293437, 'bagging_temperature': 0.5517318159431245, 'leaf_estimation_iterations': 2}. Best is trial 0 with value: 0.8317919801478025.


Best trial: 0. Best value: 0.831792:  20%|██        | 2/10 [2:37:40<10:30:43, 4730.48s/it, 9460.96/7200 seconds]
2025-04-21 14:38:15 - INFO - Обучение модели на лучших параметрах...
Подготовка отрицательных примеров: 100%|██████████| 601/601 [04:13<00:00,  2.37it/s]


0:	test: 0.7768302	best: 0.7768302 (0)	total: 6.48s	remaining: 1h 47m 55s
1:	test: 0.7768302	best: 0.7768302 (0)	total: 12.9s	remaining: 1h 47m 21s
2:	test: 0.7768302	best: 0.7768302 (0)	total: 19s	remaining: 1h 45m 24s
3:	test: 0.7768302	best: 0.7768302 (0)	total: 22.2s	remaining: 1h 32m 9s
4:	test: 0.7768302	best: 0.7768302 (0)	total: 25.6s	remaining: 1h 24m 45s
5:	test: 0.7768302	best: 0.7768302 (0)	total: 28.8s	remaining: 1h 19m 32s
6:	test: 0.7768302	best: 0.7768302 (0)	total: 32.1s	remaining: 1h 15m 53s
7:	test: 0.7768302	best: 0.7768302 (0)	total: 35.3s	remaining: 1h 12m 59s
8:	test: 0.7768302	best: 0.7768302 (0)	total: 38.6s	remaining: 1h 10m 54s
9:	test: 0.7768302	best: 0.7768302 (0)	total: 41.9s	remaining: 1h 9m 12s
10:	test: 0.7768302	best: 0.7768302 (0)	total: 45.2s	remaining: 1h 7m 42s
11:	test: 0.7768302	best: 0.7768302 (0)	total: 48.6s	remaining: 1h 6m 37s
12:	test: 0.7768302	best: 0.7768302 (0)	total: 51.8s	remaining: 1h 5m 34s
13:	test: 0.7768302	best: 0.7768302 (0)	to

2025-04-21 15:27:34 - INFO - 
Лучшие параметры:
2025-04-21 15:27:34 - INFO - learning_rate: 0.004684283500572371
2025-04-21 15:27:34 - INFO - depth: 6
2025-04-21 15:27:34 - INFO - l2_leaf_reg: 5.36104374072008e-06
2025-04-21 15:27:34 - INFO - random_strength: 0.007652648646042903
2025-04-21 15:27:34 - INFO - bagging_temperature: 0.9487648394691982
2025-04-21 15:27:34 - INFO - leaf_estimation_iterations: 9
2025-04-21 15:27:34 - INFO - 
Метрики на лучших параметрах:
2025-04-21 15:27:34 - INFO - ndcg_10: 0.8393
2025-04-21 15:27:34 - INFO - precision_10: 0.1210
2025-04-21 15:27:34 - INFO - recall_10: 0.3156
2025-04-21 15:27:34 - INFO - diversity_10: 0.0052
2025-04-21 15:27:34 - INFO - novelty_10: 0.3672
2025-04-21 15:27:34 - INFO - serendipity_10: 0.0039
2025-04-21 15:27:34 - INFO - ndcg_100: 0.8272
2025-04-21 15:27:34 - INFO - precision_100: 0.0697
2025-04-21 15:27:34 - INFO - recall_100: 0.3538
2025-04-21 15:27:34 - INFO - diversity_100: 0.0225
2025-04-21 15:27:34 - INFO - novelty_100: 0

## Модель 3: XGBoost (Градиентный бустинг над решающими деревьями)

In [5]:
train_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_train.csv"
)
test_data = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_test.csv"
)
candidates = pd.read_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_als_recommendations.csv"
)

item_categories = {
    row["product_id"]: row["product_group"] for _, row in train_data.iterrows()
}

In [6]:
class XGBoostRanker:
    """
    Ранжировщик на основе XGBoost.

    Модель использует градиентный бустинг для ранжирования товаров-кандидатов
    на основе исторических данных о покупках и характеристик пользователей.

    Attrs:
    ----------
    model : xgb.XGBRanker
        Модель XGBoost для ранжирования
    user_encoder : LabelEncoder
        Энкодер для преобразования ID пользователей в индексы
    item_encoder : LabelEncoder
        Энкодер для преобразования ID товаров в индексы
    product_group_encoder : LabelEncoder
        Энкодер для преобразования групп товаров в индексы
    feature_columns : List[str]
        Список колонок, используемых как признаки для обучения
    """

    def __init__(
        self,
        learning_rate: float = 0.1,
        max_depth: int = 6,
        n_estimators: int = 100,
        min_child_weight: float = 1.0,
        subsample: float = 1.0,
        colsample_bytree: float = 1.0,
        gamma: float = 0.0,
        random_state: int = 42,
    ):
        """
        Инициализация ранжировщика.

        Args:
        ----------
        learning_rate : float, по умолчанию 0.1
            Скорость обучения модели
        max_depth : int, по умолчанию 6
            Максимальная глубина деревьев
        n_estimators : int, по умолчанию 100
            Количество деревьев в ансамбле
        min_child_weight : float, по умолчанию 1.0
            Минимальная сумма весов всех наблюдений, необходимая для дочернего узла
        subsample : float, по умолчанию 1.0
            Доля выборки для обучения каждого дерева
        colsample_bytree : float, по умолчанию 1.0
            Доля признаков для обучения каждого дерева
        gamma : float, по умолчанию 0.0
            Минимальное уменьшение потерь, необходимое для создания нового разбиения
        random_state : int, по умолчанию 42
            Seed для воспроизводимости результатов
        """
        self.model = xgb.XGBRanker(
            objective='rank:ndcg',
            learning_rate=learning_rate,
            max_depth=max_depth,
            n_estimators=n_estimators,
            min_child_weight=min_child_weight,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            gamma=gamma,
            random_state=random_state,
            tree_method='hist',
            ndcg_exp_gain=False,
        )

        self.user_encoder = LabelEncoder()
        self.item_encoder = LabelEncoder()
        self.product_group_encoder = LabelEncoder()

        # Базовые признаки пользователя
        self.user_features = [
            "buyer_sms_allowed",
            "buyer_emails_allowed",
            "buyer_account_status_loyal",
            "buyer_account_status_unregistered",
            "buyer_account_status_new",
            "buyer_account_status_potential",
            "buyer_account_status_lost",
            "buyer_account_status_sleeping",
            "buyer_is_female",
            "buyer_age",
        ]

        # Признаки для обучения будут дополнены в методе fit
        self.feature_columns = None

    def _prepare_user_features(
        self, data: pd.DataFrame, user_col: str = "buyer_id"
    ) -> pd.DataFrame:
        """
        Подготовка признаков пользователей.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с данными пользователей
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей

        Returns:
        ----------
        pd.DataFrame
            Датафрейм с подготовленными признаками пользователей
        """
        return data.groupby(user_col)[self.user_features].first().reset_index()

    def _prepare_item_features(
        self, data: pd.DataFrame, item_col: str = "product_id"
    ) -> pd.DataFrame:
        """
        Подготовка признаков товаров.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с данными о товарах
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        pd.DataFrame
            Датафрейм с подготовленными признаками товаров
        """
        # Агрегируем статистики по товарам
        item_stats = (
            data.groupby(item_col)
            .agg(
                {
                    "product_count": ["mean", "sum"],
                    "product_sum": ["mean", "sum"],
                    "product_group": "first",
                }
            )
            .reset_index()
        )

        # Переименовываем колонки
        item_stats.columns = [
            item_col,
            "avg_product_count",
            "total_product_count",
            "avg_product_sum",
            "total_product_sum",
            "product_group",
        ]

        # Кодируем группы товаров
        item_stats["product_group_encoded"] = self.product_group_encoder.transform(
            item_stats["product_group"]
        )

        return item_stats

    def _prepare_user_item_features(
        self,
        data: pd.DataFrame,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
    ) -> pd.DataFrame:
        """
        Подготовка признаков взаимодействия пользователь-товар.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с данными о взаимодействиях
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        pd.DataFrame
            Датафрейм с признаками взаимодействий
        """
        # Агрегируем статистики по парам пользователь-товар
        user_item_stats = (
            data.groupby([user_col, item_col])
            .agg(
                {
                    "product_count": ["count", "mean", "sum"],
                    "product_sum": ["mean", "sum"],
                    "order_date": "max",  # последняя дата покупки
                }
            )
            .reset_index()
        )

        # Переименовываем колонки
        user_item_stats.columns = [
            user_col,
            item_col,
            "interaction_count",
            "avg_purchase_count",
            "total_purchase_count",
            "avg_purchase_sum",
            "total_purchase_sum",
            "last_purchase_date",
        ]

        # Преобразуем даты в datetime
        user_item_stats["last_purchase_date"] = pd.to_datetime(
            user_item_stats["last_purchase_date"]
        )

        # Добавляем признак времени с последней покупки
        current_date = user_item_stats["last_purchase_date"].max()
        user_item_stats["days_since_last_purchase"] = (
            current_date - user_item_stats["last_purchase_date"]
        ).dt.days

        return user_item_stats

    def fit(
        self,
        data: pd.DataFrame,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
    ) -> "XGBoostRanker":
        """
        Обучение ранжировщика на исторических данных.

        Args:
        ----------
        data : pd.DataFrame
            Датафрейм с историческими данными о покупках
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        self : XGBoostRanker
            Обученный объект
        """
        # Кодируем ID пользователей и товаров
        self.user_encoder.fit(data[user_col].unique())
        self.item_encoder.fit(data[item_col].unique())
        self.product_group_encoder.fit(data["product_group"].unique())

        # Подготавливаем признаки
        user_features = self._prepare_user_features(data, user_col)
        item_features = self._prepare_item_features(data, item_col)
        user_item_features = self._prepare_user_item_features(data, user_col, item_col)

        # Объединяем все признаки
        features = user_item_features.merge(
            user_features, on=user_col, how="left"
        ).merge(item_features, on=item_col, how="left")

        # Добавляем закодированные ID
        features["user_encoded"] = self.user_encoder.transform(features[user_col])
        features["item_encoded"] = self.item_encoder.transform(features[item_col])

        # Определяем список признаков для обучения
        self.feature_columns = [
            "user_encoded",
            "item_encoded",
            "interaction_count",
            "avg_purchase_count",
            "total_purchase_count",
            "avg_purchase_sum",
            "total_purchase_sum",
            "days_since_last_purchase",
            "avg_product_count",
            "total_product_count",
            "avg_product_sum",
            "total_product_sum",
            "product_group_encoded",
        ] + self.user_features

        # Готовим данные для обучения
        X = features[self.feature_columns]
        y = features[
            "interaction_count"
        ]  # используем количество взаимодействий как целевую переменную
        groups = features.groupby(user_col).size().values

        # Обучаем модель
        self.model.fit(X, y, group=groups, verbose=True)

        return self

    def rank(
        self,
        candidates: pd.DataFrame,
        train_data: pd.DataFrame,
        top_n: int = 10,
        user_col: str = "buyer_id",
        item_col: str = "product_id",
    ) -> pd.DataFrame:
        """
        Ранжирование кандидатов на основе XGBoost.

        Args:
        ----------
        candidates : pd.DataFrame
            Датафрейм с парами пользователь-товар для ранжирования
        train_data : pd.DataFrame
            Датафрейм с историческими данными для расчета признаков
        top_n : int, по умолчанию 10
            Количество рекомендуемых товаров для каждого пользователя
        user_col : str, по умолчанию 'buyer_id'
            Название столбца с идентификаторами пользователей
        item_col : str, по умолчанию 'product_id'
            Название столбца с идентификаторами товаров

        Returns:
        ----------
        pd.DataFrame
            Датафрейм с ранжированными рекомендациями и их скорами
        """
        if not hasattr(self, "model") or self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit()")

        # Фильтруем только известные пользователи и товары
        known_users_mask = candidates[user_col].isin(self.user_encoder.classes_)
        known_items_mask = candidates[item_col].isin(self.item_encoder.classes_)
        valid_candidates = candidates[known_users_mask & known_items_mask].copy()

        if valid_candidates.empty:
            return pd.DataFrame(columns=[user_col, item_col, "score"])

        # Подготавливаем признаки
        user_features = self._prepare_user_features(train_data, user_col)
        item_features = self._prepare_item_features(train_data, item_col)
        user_item_features = self._prepare_user_item_features(
            train_data, user_col, item_col
        )

        # Объединяем признаки с кандидатами
        features = (
            valid_candidates.merge(user_features, on=user_col, how="left")
            .merge(item_features, on=item_col, how="left")
            .merge(user_item_features, on=[user_col, item_col], how="left")
        )

        # Заполняем пропущенные значения
        features = features.fillna(
            {
                "interaction_count": 0,
                "avg_purchase_count": 0,
                "total_purchase_count": 0,
                "avg_purchase_sum": 0,
                "total_purchase_sum": 0,
                "days_since_last_purchase": (
                    features["days_since_last_purchase"].max()
                    if not pd.isna(features["days_since_last_purchase"]).all()
                    else 365
                ),  # если нет данных, предполагаем год
            }
        )

        # Кодируем ID
        features["user_encoded"] = self.user_encoder.transform(features[user_col])
        features["item_encoded"] = self.item_encoder.transform(features[item_col])

        # Получаем предсказания
        X = features[self.feature_columns]
        scores = self.model.predict(X)

        # Формируем результат
        recommendations = features[[user_col, item_col]].copy()
        recommendations["score"] = scores

        # Сортируем и отбираем top_n
        recommendations = recommendations.sort_values(
            by=[user_col, "score"], ascending=[True, False]
        )
        recommendations = recommendations.groupby(user_col).head(top_n)

        return recommendations
    
    def save_model(
        self, 
        model_path: str, 
        export_parameters: bool = True
    ) -> None:
        """
        Сохранение модели и её параметров.

        Args:
            model_path: путь для сохранения модели
            export_parameters: сохранять ли дополнительные параметры модели
        """
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала вызовите метод fit()")

        # Создаем директорию если её нет
        os.makedirs(os.path.dirname(model_path), exist_ok=True)

        # Сохраняем модель XGBoost
        self.model.save_model(model_path)

        if export_parameters:
            # Сохраняем дополнительные параметры модели
            parameters = {
                'user_encoder': self.user_encoder,
                'item_encoder': self.item_encoder,
                'product_group_encoder': self.product_group_encoder,
                'user_features': self.user_features,
                'feature_columns': self.feature_columns
            }

            # Путь для сохранения параметров
            params_path = f"{os.path.splitext(model_path)[0]}_params.joblib"

            joblib.dump(parameters, params_path)

    @classmethod
    def load_model(
        cls, 
        model_path: str, 
        load_parameters: bool = True
    ) -> "XGBoostRanker":
        """
        Загрузка сохранённой модели и её параметров.

        Args:
            model_path: путь к сохранённой модели
            load_parameters: загружать ли дополнительные параметры модели

        Returns:
            XGBoostRanker: загруженная модель
        """
        # Создаем экземпляр класса
        ranker = cls()

        # Загружаем модель XGBoost
        ranker.model = xgb.XGBRanker()
        ranker.model.load_model(model_path)

        if load_parameters:
            # Путь к файлу с параметрами
            params_path = f"{os.path.splitext(model_path)[0]}_params.joblib"

            if os.path.exists(params_path):
                parameters = joblib.load(params_path)

                # Загружаем параметры
                ranker.user_encoder = parameters['user_encoder']
                ranker.item_encoder = parameters['item_encoder']
                ranker.product_group_encoder = parameters['product_group_encoder']
                ranker.user_features = parameters['user_features']
                ranker.feature_columns = parameters['feature_columns']
            else:
                logging.warning(
                    f"Файл с параметрами {params_path} не найден. "
                    "Загружена только модель без дополнительных параметров."
                )

        return ranker

In [29]:
def optimize_xgboost_parameters(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    candidates: pd.DataFrame,
    metrics_calculator: MetricsCalculator,
    item_categories: Dict[str, str] = None,
    n_trials: int = 10,
    k_values: List[int] = [10, 100, 1000],
    random_state: int = 42,
    timeout: int = 7200,
) -> Tuple[Dict[str, float], Dict[str, float]]:
    """
    Оптимизация гиперпараметров XGBoostRanker с помощью Optuna.

    Args:
    ----------
    train_data : pd.DataFrame
        Датафрейм с историческими данными о покупках для обучения
    test_data : pd.DataFrame
        Датафрейм с историческими данными о покупках для тестирования
    candidates : pd.DataFrame
        Датафрейм с парами пользователь-товар для ранжирования
    metrics_calculator : MetricsCalculator
        Калькулятор метрик
    item_categories : Dict[str, str], по умолчанию None
        Словарь соответствия товаров и их категорий {item_id: category_id}
    n_trials : int, по умолчанию 10
        Количество итераций оптимизации
    k_values : List[int], по умолчанию [10, 100, 1000]
        Список значений K для расчета метрик
    random_state : int, по умолчанию 42
        Seed для воспроизводимости результатов
    timeout : int, по умолчанию 42
        Максимальное время оптимизации в секундах

    Returns:
    ----------
    Tuple[Dict[str, float], Dict[str, float]]
        Кортеж из двух словарей:
        1. Лучшие найденные гиперпараметры
        2. Значения метрик на тестовой выборке для лучших параметров
    """

    def objective(trial: Trial) -> float:
        """
        Целевая функция для оптимизации.

        Args:
        ----------
        trial : Trial
            Объект Trial из Optuna для предложения параметров

        Returns:
        ----------
        float
            Среднее значение NDCG@K на тестовой выборке
        """
        # Определяем пространство поиска гиперпараметров
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "n_estimators": trial.suggest_int("n_estimators", 50, 300),
            "min_child_weight": trial.suggest_float(
                "min_child_weight", 1e-3, 10.0, log=True
            ),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "gamma": trial.suggest_float("gamma", 1e-3, 10.0, log=True),
            "random_state": random_state,
        }

        # Создаем и обучаем модель
        model = XGBoostRanker(**params)
        model.fit(train_data)

        # Получаем рекомендации
        recommendations = model.rank(
            candidates=candidates, train_data=train_data, top_n=max(k_values)
        )

        # Преобразуем рекомендации в словарь для расчета метрик
        recommendations_dict = (
            recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
        )

        # Рассчитываем метрики
        metrics = metrics_calculator.calculate(
            recommendations=recommendations_dict,
            train_data=train_data,
            test_data=test_data,
        )

        return metrics[f"ndcg_{max(k_values)}"]

    # Создаем исследование Optuna
    study_name = f"xgboost_optimization_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    study = optuna.create_study(
        direction="maximize",
        study_name=study_name,
        sampler=optuna.samplers.TPESampler(seed=random_state),
    )

    # Запускаем оптимизацию
    study.optimize(
        objective, n_trials=n_trials, timeout=timeout, show_progress_bar=True
    )

    # Получаем лучшие параметры
    best_params = study.best_params

    # Обучаем модель на полных данных с лучшими параметрами
    best_model = XGBoostRanker(
        learning_rate=best_params["learning_rate"],
        max_depth=best_params["max_depth"],
        n_estimators=best_params["n_estimators"],
        min_child_weight=best_params["min_child_weight"],
        subsample=best_params["subsample"],
        colsample_bytree=best_params["colsample_bytree"],
        gamma=best_params["gamma"],
        random_state=random_state,
    )
    best_model.fit(train_data)

    # Получаем рекомендации
    recommendations = best_model.rank(
        candidates=candidates, train_data=train_data, top_n=max(k_values)
    )

    recommendations_dict = (
        recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
    )

    # Рассчитываем финальные метрики
    final_metrics = metrics_calculator.calculate(
        recommendations=recommendations_dict,
        train_data=train_data,
        test_data=test_data,  # используем тестовые данные для финальной оценки
        item_categories=item_categories,
    )

    return best_params, final_metrics

In [30]:
if __name__ == "__main__":
    n_samples = 10000
    # buyer_ids = test_data["buyer_id"].unique()[:n_samples]
    buyer_ids = test_data["buyer_id"].unique()

    temp_candidates = candidates[candidates["buyer_id"].isin(buyer_ids)].copy()

    product_ids = temp_candidates["product_id"].unique()
    temp_train_data = train_data[
        (train_data["buyer_id"].isin(buyer_ids))
        & (train_data["product_id"].isin(product_ids))
    ].copy()

    temp_test_data = test_data[
        (test_data["buyer_id"].isin(buyer_ids))
        & (test_data["product_id"].isin(product_ids))
    ].copy()

    metrics_calculator = MetricsCalculator([10, 100, 1000])

    best_params, best_metrics = optimize_xgboost_parameters(
        train_data=temp_train_data,
        test_data=temp_test_data,
        candidates=temp_candidates,
        metrics_calculator=metrics_calculator,
        item_categories=item_categories,
        n_trials=10,
        timeout=7200,
    )

    logging.info(f"Лучшие параметры: {best_params}")
    logging.info("Результаты:")
    for k in metrics_calculator.k_values:
        logging.info(f"Метрики для K={k}:")
        logging.info(f"NDCG@{k}: {best_metrics[f'ndcg_{k}']:.4f}")
        logging.info(f"Precision@{k}: {best_metrics[f'precision_{k}']:.4f}")
        logging.info(f"Recall@{k}: {best_metrics[f'recall_{k}']:.4f}")
        logging.info(f"Diversity@{k}: {best_metrics[f'diversity_{k}']:.4f}")
        logging.info(f"Novelty@{k}: {best_metrics[f'novelty_{k}']:.4f}")
        logging.info(f"Serendipity@{k}: {best_metrics[f'serendipity_{k}']:.4f}")
        logging.info("--------------------------------")

[I 2025-04-26 08:25:01,049] A new study created in memory with name: xgboost_optimization_20250426_082501
Best trial: 0. Best value: 0.832345:  10%|█         | 1/10 [05:48<52:13, 348.11s/it, 348.11/7200 seconds]

[I 2025-04-26 08:30:49,160] Trial 0 finished with value: 0.8323453801529145 and parameters: {'learning_rate': 0.008468008575248327, 'max_depth': 10, 'n_estimators': 233, 'min_child_weight': 0.24810409748678125, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.0017073967431528124}. Best is trial 0 with value: 0.8323453801529145.


Best trial: 0. Best value: 0.832345:  20%|██        | 2/10 [11:14<44:44, 335.55s/it, 674.88/7200 seconds]

[I 2025-04-26 08:36:15,923] Trial 1 finished with value: 0.8323453801529145 and parameters: {'learning_rate': 0.13983740016490973, 'max_depth': 7, 'n_estimators': 227, 'min_child_weight': 0.0012087541473056963, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'gamma': 0.0070689749506246055}. Best is trial 0 with value: 0.8323453801529145.


Best trial: 0. Best value: 0.832345:  30%|███       | 3/10 [16:38<38:30, 330.10s/it, 998.49/7200 seconds]

...

Best trial: 0. Best value: 0.832345:  90%|█████████ | 9/10 [49:41<05:28, 328.75s/it, 2981.90/7200 seconds]

[I 2025-04-26 09:14:42,949] Trial 8 finished with value: 0.8323453801529145 and parameters: {'learning_rate': 0.0016565580440884786, 'max_depth': 4, 'n_estimators': 61, 'min_child_weight': 0.02001342062287998, 'subsample': 0.6943386448447411, 'colsample_bytree': 0.6356745158869479, 'gamma': 2.0651425578959257}. Best is trial 0 with value: 0.8323453801529145.


Best trial: 0. Best value: 0.832345: 100%|██████████| 10/10 [55:25<00:00, 332.59s/it, 3325.86/7200 seconds]


[I 2025-04-26 09:20:26,906] Trial 9 finished with value: 0.8323453801529145 and parameters: {'learning_rate': 0.0076510536667541975, 'max_depth': 5, 'n_estimators': 186, 'min_child_weight': 0.0036618192203924276, 'subsample': 0.9010984903770198, 'colsample_bytree': 0.5372753218398854, 'gamma': 8.862326508576253}. Best is trial 0 with value: 0.8323453801529145.


2025-04-26 09:28:12 - INFO - Лучшие параметры: {'learning_rate': 0.008468008575248327, 'max_depth': 10, 'n_estimators': 233, 'min_child_weight': 0.24810409748678125, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.0017073967431528124}
2025-04-26 09:28:12 - INFO - Результаты:
2025-04-26 09:28:12 - INFO - Метрики для K=10:
2025-04-26 09:28:12 - INFO - NDCG@10: 0.8398
2025-04-26 09:28:12 - INFO - Precision@10: 0.0997
2025-04-26 09:28:12 - INFO - Recall@10: 0.2215
2025-04-26 09:28:12 - INFO - Diversity@10: 0.0033
2025-04-26 09:28:12 - INFO - Novelty@10: 0.7153
2025-04-26 09:28:12 - INFO - Serendipity@10: 0.0028
2025-04-26 09:28:12 - INFO - --------------------------------
2025-04-26 09:28:12 - INFO - Метрики для K=100:
2025-04-26 09:28:12 - INFO - NDCG@100: 0.8277
2025-04-26 09:28:12 - INFO - Precision@100: 0.0563
2025-04-26 09:28:12 - INFO - Recall@100: 0.2472
2025-04-26 09:28:12 - INFO - Diversity@100: 0.0123
2025-04-26 09:28:12 - INFO - Novelty@100: 0

## Сохранение лучших моделей

In [9]:
logging.info("Обучение модели MatrixFactorizationRanker...")
model_params = {
    "n_components": 184,
    "learning_rate": 0.36596795884334166,
    "loss": 'warp-kos',
    "n_epochs": 23,
    "random_state": 42,
}

mf_ranker = MatrixFactorizationRanker(**model_params)
mf_ranker.fit(train_data)

logging.info("Сохранение модели MatrixFactorizationRanker...")
model_path = (
    f"{DATA_PATH}/models/matrix_factorization_ranker_{ORGANIZATION_ID}_{PROCESSING_DATE}.joblib"
)
mf_ranker.save_model(model_path)

logging.info("Загрузка модели MatrixFactorizationRanker...")
mf_ranker = MatrixFactorizationRanker.load_model(model_path)

logging.info("Генерация финальных рекомендаций...")
ranked_recommendations = mf_ranker.rank(
    candidates=candidates, top_n=1000,
)

logging.info("Сохранение рекомендаций...")
ranked_recommendations.to_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_mf_recommendations.csv", index=False
)

recommendations_dict = (
    ranked_recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
)

logging.info("Расчет финальных метрик...")
metrics_calculator = MetricsCalculator([10, 100, 1000])
final_metrics = metrics_calculator.calculate(
    recommendations=recommendations_dict,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"NDCG@{k}: {final_metrics[f'ndcg_{k}']:.4f}")
    logging.info(f"Precision@{k}: {final_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {final_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {final_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {final_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {final_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

2025-04-27 10:52:55 - INFO - Обучение модели MatrixFactorizationRanker...
Epoch: 100%|██████████| 23/23 [08:19<00:00, 21.74s/it]
2025-04-27 11:01:18 - INFO - Сохранение модели MatrixFactorizationRanker...
2025-04-27 11:01:19 - INFO - Загрузка модели MatrixFactorizationRanker...
2025-04-27 11:01:19 - INFO - Генерация финальных рекомендаций...
2025-04-27 11:02:50 - INFO - Сохранение рекомендаций...
2025-04-27 11:09:53 - INFO - Расчет финальных метрик...
2025-04-27 11:14:02 - INFO - Результаты:
2025-04-27 11:14:02 - INFO - Метрики для K=10:
2025-04-27 11:14:02 - INFO - NDCG@10: 0.8367
2025-04-27 11:14:02 - INFO - Precision@10: 0.0021
2025-04-27 11:14:02 - INFO - Recall@10: 0.0037
2025-04-27 11:14:02 - INFO - Diversity@10: 0.0055
2025-04-27 11:14:02 - INFO - Novelty@10: 0.9963
2025-04-27 11:14:02 - INFO - Serendipity@10: 0.0001
2025-04-27 11:14:02 - INFO - --------------------------------
2025-04-27 11:14:02 - INFO - Метрики для K=100:
2025-04-27 11:14:02 - INFO - NDCG@100: 0.8246
2025-04-

In [10]:
logging.info("Обучение модели CatBoostRanker...")
model_params = {
    "learning_rate": 0.004684283500572371,
    "iterations": 1000,
    "depth": 6,
    "l2_leaf_reg": 5.36104374072008e-06,
    "random_seed": 42,
    "thread_count": -1,
    "verbose": True,
    "task_type": "CPU",
    "loss_function": "Logloss",
    "eval_metric": "NDCG",
    "early_stopping_rounds": 50,
    "random_strength": 0.007652648646042903,
    "bagging_temperature": 0.9487648394691982,
    "leaf_estimation_iterations": 9,
}

cat_boost_ranker = CatBoostRanker(
    learning_rate=model_params["learning_rate"],
    iterations=model_params["iterations"],
    depth=model_params["depth"],
    l2_leaf_reg=model_params["l2_leaf_reg"],
    random_seed=42,
    thread_count=model_params["thread_count"],
    verbose=model_params["verbose"],
)
cat_boost_ranker.model_params.update(model_params)
cat_boost_ranker.fit(train_data, candidates)

logging.info("Сохранение модели CatBoostRanker...")
model_path = (
    f"{DATA_PATH}/models/cat_boost_ranker_{ORGANIZATION_ID}_{PROCESSING_DATE}.cbm"
)
cat_boost_ranker.save_model(model_path)

logging.info("Загрузка модели CatBoostRanker...")
cat_boost_ranker = CatBoostRanker.load_model(model_path)

logging.info("Генерация финальных рекомендаций...")
ranked_recommendations = cat_boost_ranker.rank(
    candidates=candidates, train_data=train_data, top_n=1000
)

recommendations_dict = (
    ranked_recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
)

logging.info("Расчет финальных метрик...")
metrics_calculator = MetricsCalculator([10, 100, 1000])
final_metrics = metrics_calculator.calculate(
    recommendations=recommendations_dict,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"NDCG@{k}: {final_metrics[f'ndcg_{k}']:.4f}")
    logging.info(f"Precision@{k}: {final_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {final_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {final_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {final_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {final_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

2025-04-27 15:41:17 - INFO - Обучение модели CatBoostRanker...
Подготовка отрицательных примеров: 100%|██████████| 601/601 [13:46<00:00,  1.38s/it]


0:	test: 0.8790034	best: 0.8790034 (0)	total: 6.77s	remaining: 1h 52m 43s
1:	test: 0.8790034	best: 0.8790034 (0)	total: 9.55s	remaining: 1h 19m 23s
2:	test: 0.8790034	best: 0.8790034 (0)	total: 16.2s	remaining: 1h 29m 42s
3:	test: 0.8790034	best: 0.8790034 (0)	total: 18.6s	remaining: 1h 17m 13s
4:	test: 0.8790034	best: 0.8790034 (0)	total: 21s	remaining: 1h 9m 29s
5:	test: 0.8790034	best: 0.8790034 (0)	total: 23.5s	remaining: 1h 4m 56s
6:	test: 0.8790034	best: 0.8790034 (0)	total: 26s	remaining: 1h 1m 27s
7:	test: 0.8790034	best: 0.8790034 (0)	total: 28.6s	remaining: 59m 12s
8:	test: 0.8860134	best: 0.8860134 (8)	total: 32.4s	remaining: 59m 24s
9:	test: 0.8860530	best: 0.8860530 (9)	total: 38.6s	remaining: 1h 3m 37s
10:	test: 0.8860526	best: 0.8860530 (9)	total: 42.4s	remaining: 1h 3m 31s
11:	test: 0.8860620	best: 0.8860620 (11)	total: 45s	remaining: 1h 1m 42s
12:	test: 0.8860583	best: 0.8860620 (11)	total: 47.5s	remaining: 1h 4s
13:	test: 0.8860441	best: 0.8860620 (11)	total: 51.3s	re

2025-04-27 17:01:31 - INFO - Сохранение модели CatBoostRanker...
2025-04-27 17:01:32 - INFO - Загрузка модели CatBoostRanker...
2025-04-27 17:01:34 - INFO - Генерация финальных рекомендаций...
IOStream.flush timed out
2025-04-27 17:05:02 - INFO - Расчет финальных метрик...
2025-04-27 17:08:34 - INFO - Результаты:
2025-04-27 17:08:34 - INFO - Метрики для K=10:
2025-04-27 17:08:34 - INFO - NDCG@10: 0.8367
2025-04-27 17:08:34 - INFO - Precision@10: 0.1200
2025-04-27 17:08:34 - INFO - Recall@10: 0.3078
2025-04-27 17:08:34 - INFO - Diversity@10: 0.0052
2025-04-27 17:08:34 - INFO - Novelty@10: 0.6922
2025-04-27 17:08:34 - INFO - Serendipity@10: 0.0013
2025-04-27 17:08:34 - INFO - --------------------------------
2025-04-27 17:08:34 - INFO - Метрики для K=100:
2025-04-27 17:08:34 - INFO - NDCG@100: 0.8246
2025-04-27 17:08:34 - INFO - Precision@100: 0.0691
2025-04-27 17:08:34 - INFO - Recall@100: 0.3458
2025-04-27 17:08:34 - INFO - Diversity@100: 0.0225
2025-04-27 17:08:34 - INFO - Novelty@100

In [11]:
logging.info("Обучение модели XGBoostRanker...")
model_params = {
    "learning_rate": 0.008468008575248327,
    "max_depth": 10,
    "n_estimators": 233,
    "min_child_weight": 0.24810409748678125,
    "subsample": 0.5780093202212182,
    "colsample_bytree": 0.5779972601681014,
    "gamma": 0.0017073967431528124,
    "random_state": 42,
}

mf_ranker = XGBoostRanker(**model_params)
mf_ranker.fit(train_data)

logging.info("Сохранение модели XGBoostRanker...")
model_path = f"{DATA_PATH}/models/xgb_ranker_{ORGANIZATION_ID}_{PROCESSING_DATE}.xgb"
mf_ranker.save_model(model_path)

logging.info("Загрузка модели XGBoostRanker...")
mf_ranker = XGBoostRanker.load_model(model_path)

logging.info("Генерация финальных рекомендаций...")
ranked_recommendations = mf_ranker.rank(
    candidates=candidates, top_n=1000, train_data=train_data
)

logging.info("Сохранение рекомендаций...")
ranked_recommendations.to_csv(
    f"{DATA_PATH}/{ORGANIZATION_ID}_{PROCESSING_DATE}_xgb_recommendations.csv", index=False
)

recommendations_dict = (
    ranked_recommendations.groupby("buyer_id")["product_id"].agg(list).to_dict()
)

logging.info("Расчет финальных метрик...")
metrics_calculator = MetricsCalculator([10, 100, 1000])
final_metrics = metrics_calculator.calculate(
    recommendations=recommendations_dict,
    train_data=train_data,
    test_data=test_data,
    item_categories=item_categories,
)

logging.info("Результаты:")
for k in metrics_calculator.k_values:
    logging.info(f"Метрики для K={k}:")
    logging.info(f"NDCG@{k}: {final_metrics[f'ndcg_{k}']:.4f}")
    logging.info(f"Precision@{k}: {final_metrics[f'precision_{k}']:.4f}")
    logging.info(f"Recall@{k}: {final_metrics[f'recall_{k}']:.4f}")
    logging.info(f"Diversity@{k}: {final_metrics[f'diversity_{k}']:.4f}")
    logging.info(f"Novelty@{k}: {final_metrics[f'novelty_{k}']:.4f}")
    logging.info(f"Serendipity@{k}: {final_metrics[f'serendipity_{k}']:.4f}")
    logging.info("--------------------------------")

2025-04-27 17:08:34 - INFO - Обучение модели XGBoostRanker...


[17:09:30] WARNING: ../src/learner.cc:767: 
Parameters: { "ndcg_exp_gain" } are not used.



2025-04-27 17:09:36 - INFO - Сохранение модели XGBoostRanker...
2025-04-27 17:09:36 - INFO - Загрузка модели XGBoostRanker...
2025-04-27 17:09:36 - INFO - Генерация финальных рекомендаций...
2025-04-27 17:13:13 - INFO - Сохранение рекомендаций...
2025-04-27 17:19:11 - INFO - Расчет финальных метрик...
2025-04-27 17:22:42 - INFO - Результаты:
2025-04-27 17:22:42 - INFO - Метрики для K=10:
2025-04-27 17:22:42 - INFO - NDCG@10: 0.8367
2025-04-27 17:22:42 - INFO - Precision@10: 0.0962
2025-04-27 17:22:42 - INFO - Recall@10: 0.2067
2025-04-27 17:22:42 - INFO - Diversity@10: 0.0046
2025-04-27 17:22:42 - INFO - Novelty@10: 0.7933
2025-04-27 17:22:42 - INFO - Serendipity@10: 0.0009
2025-04-27 17:22:42 - INFO - --------------------------------
2025-04-27 17:22:42 - INFO - Метрики для K=100:
2025-04-27 17:22:42 - INFO - NDCG@100: 0.8246
2025-04-27 17:22:42 - INFO - Precision@100: 0.0538
2025-04-27 17:22:42 - INFO - Recall@100: 0.2220
2025-04-27 17:22:42 - INFO - Diversity@100: 0.0142
2025-04-27 

## Сравнение моделей

### Лучшие гиперпараметры моделей

**LightFM (Матричная факторизация)**:
```json
{
    "n_components": 184,
    "learning_rate": 0.36596795884334166,
    "loss": "warp-kos",
    "n_epochs": 23,
}
```

**CatBoost (Градиентный бустинг над решающими деревьями)**:
```json
{
    "learning_rate": 0.004684283500572371, 
    "depth": 6, 
    "l2_leaf_reg": 5.36104374072008e-06,
    "random_strength": 0.007652648646042903,
    "bagging_temperature": 0.9487648394691982,
    "leaf_estimation_iterations": 9
}
```

**XGBoost (Градиентный бустинг над решающими деревьями)**
```json
{
    "learning_rate": 0.008468008575248327,
    "max_depth": 10,
    "n_estimators": 233,
    "min_child_weight": 0.24810409748678125,
    "subsample": 0.5780093202212182,
    "colsample_bytree": 0.5779972601681014,
    "gamma": 0.0017073967431528124,
}
```


### Метрики работы моделей

| Метрика  | LightFM  | CatBoost  | XGBRanker  |
|---|---|---|---|
|  Время обучения |  9 мин. |  60 мин. |  1 мин. |
|  Время получения предсказаний |  2 мин. |  4 мин. |  4 мин. |
|  NDCG@10 |  0.8367 |  0.8367 |  0.8367 |
|  NDCG@100 |  0.8246 |  0.8246 |  0.8246 |
|  NDCG@1000 |  0.8291 |  0.8291 |  0.8291 |
|  Precision@10 |  0.0021 |  0.1180 |  0.0962 |
|  Precision@100 |  0.0018 |  0.0681 |  0.0538 |
|  Precision@1000 |  0.0024 |  0.0466 |  0.0371 |
|  Recall@10 |  0.0037 |  0.3042 |  0.2067 |
|  Recall@100 |  0.0142 |  0.3436 |  0.2220 |
|  Recall@1000 |  0.2632 |  0.4828 |  0.4017 |
|  Diversity@10 |  0.0055 |  0.0053 |  0.0046 |
|  Diversity@100 |  0.0224 |  0.0225 |  0.0142 |
|  Diversity@1000 |  0.0777 |  0.0777 |  0.0722 |
|  Novelty@10 |  0.9963 |  0.6958 |  0.7933 |
|  Novelty@100 |  0.9858 |  0.6564 |  0.7780 |
|  Novelty@1000 |  0.7368 |  0.5172 |  0.5983 |
|  Serendipity@10 |  0.0001 |  0.0011 |  0.0009 |
|  Serendipity@100 |  0.0001 |  0.0006 |  0.0005 |
|  Serendipity@1000 |  0.0001 |  0.0004 |  0.0003 |

**Сравнение временных затрат производилось на одинаковых датасетах и одинаковом оборудовании на платформе Intel Ice Lake с 32 vCPU и 256 Гб RAM**


## Итоги

В ходе работы было произведено:
- анализ подходящих моделей и алгоритмов машинного обучения для ранжирования сгенерированных кандидатов
- реализована модель матричной факторизации - LightFM
- реализована модель градиентного бустинга над решающими деревьями - CatBoost
- реализована модель градиентного бустинга над решающими деревьями - XGBoost
- проведено обучение моделей
- выполнен анализ результатов работы моделей

Согласно результатам работы моделей, значения ключевой метрики - NDCG - одинаковое у всех.

CatBoost был отброшен в следствии долгого времени обучения.

Выбор между LightFM и XGBRanker пал в пользу XGBRanker - быстрее обучается, предоставляет достаточно новые для пользователя рекомендации, но не столь экстремально новые, как LightFM.